<a class="anchor" id="top"></a>
# Table of Contents:

1. [Arrays and Strings](#arrays)
2. [Linked Lists](#linkedlists)
3. [Stacks and Queues](#stacks)
4. [Trees and Graphs](#trees)
5. [Bit Manipulation](#bits)
6. [Object-Oriented Design](#ood)
7. [Recursion and Dynamic Programming](#dp)
8. [System Design and Scalability](#system)
9. [Sorting and Searching](#sorts)

<a class="anchor" id="arrays"></a>
# Arrays and Strings

### General stuff:
* use a bit vector to speed things up if you're looking for unique chars or integers
    - assume bit vector = 000000000
    - then convert chars to ASCII codes and create a bit mask where mask = 1 << char.charCodeAt() - 'a'.charCodeAt()
        - the 'a'.charCodeAt() is used or else it will cause int. overflow
    - then do bitVector & mask.
        - if we get a non-zero value, then we have a duplicate
        - else, we do bitVector |= mask to set the bitVector at the masks' position
* if dealing with string rotations, e.g. we have string1 and string2 and we want to see if string2 = rotation of string1, then:
    - concat string2 with itself
        - so if string2 = msonsa
        - string2 + string2 = msonsamson
    - then we find string1 in the concated version
        - so 'msonsamson'.indexOf('samson') !== -1

### Things to Ask:
1. if working with strings, is the char set ASCII or Unicode?
2. do we have to do input validation?
3. are strings case sensitive? is 'a' different from 'A'
    - this is to determine if we should use .toLowerCase() on string first and then proceed with the algorithm

### [Go back to top](#top)

## 1.1 Is Unique (string)
* Implement an algorithm to determine if a string has all unique characters. What if you cannot use additional data structures?

In [2]:
// use of a hash table to solve it
// time: O(n), space = O(n)

function isUnique(str) {
    let table = {};
    for(let i = 0; i < str.length; i++) {
        let char = str[i];
        if(table[char]) {
            return false;
        }
        table[char] = true;
    }
    return true;
}

console.log(isUnique('hi'));
console.log(isUnique('hello world'));

true
false


In [13]:
// using a bit vector

function isUniqueBits(str) {
    
    // assuming ASCII chars, then there are only 128 possible selections
    // if str.length > 128, then that means there is at least 1 duplicate
    
    if(str.length > 128) {
        return false;
    }
    
    // 0000000 etc
    let bitVector = 0;
    const offset = 'a'.charCodeAt(); // offset = 97 for ASCII
    for(let i = 0; i < str.length; i++) {
        
        // this is so that the charCode is not too big
        let val = str[i].charCodeAt() - offset;
        
        // if val = 3
        // mask = 00001000
        let mask = 1 << val;
        
        // if our bit vector = 10001000
        // and mask = 00001000
        // and we do bitVector & mask
        // then if we get a non-zero number, we know we
        // have a duplicate
        // this is b/c if all the charCodes are unique
        // then bitVector & mask would yield 0000000
        if(bitVector & mask) {
            return false;
        }
        
        // sets the bitVector at the charCode position
        // so if our bitVector = 10000000
        // and our mask = 00001000
        // then bitVector |= mask
        // would yield 10001000
        bitVector |= mask;
    }
    return true;
}

console.log(isUniqueBits('hi'));
console.log(isUniqueBits('hello world'));

true
false


## 1.9 String Rotation
* Assume you have a method isSubstring which checks if one word is a substring of another. given two strings, s1 and s2, write code to check if s2 is a rotation of s1 using only one call to isSubstring (e.g. "waterbottle" is a rotation of "erbottlewat").

In [19]:
function isSubString(s1, s2) {
    // if they aren't the same length
    // they can't be rotations
    if(s1.length !== s2.length) {
        return false;
    }
    
    let newString = s2 + s2;
    console.log(newString)
    
    // can also use newString.includes(s1)
    // but indexOf is faster on most browsers
    return newString.indexOf(s1) !== -1;
}

console.log(isSubString('waterbottle', 'erbottlewat')); //true
console.log(isSubString('samson', 'nguyen')); // false

erbottlewaterbottlewat
true
nguyennguyen
false


## Sliding Window Technique
* used to reduce a nested loop O(n$^{2}$) algorithm down to O(n)
* technique used on strings and arrays usually where we want to look for contiguous subarrays of something

***
Example: 
* given array of size 'n', we want to calculate max sum of 'k' consecutive elements in the array
    - input: arr = [100, 200, 300, 400], k = 2
    - output: 700 ([300, 400])
    - input: arr = [1, 4, 2, 10, 23, 3, 1, 0, 20], k = 4
    - output: 39 ([4, 2, 10, 23])
    - input: arr = [2, 3], k = 3
    - output = invalid (b/c k > arr.length)

In [13]:
// brute-force approach
// O(k * n)
// k = # of elemtns in the contiguous subarray
// that we want

function maxSum(arr, k) {
    let max_sum = Number.MIN_VALUE;
    
    // goes through each element in the arr
    for(let i = 0; i < arr.length - k + 1; i++) {
        let current_sum = 0;
        
        // then checks i + 0, i + 1, i + 2, i + 3
        // and adds them to current_sum
        for(let j = 0; j < k; j++) {
            current_sum = current_sum + arr[i + j];
            
            // updates max_sum if current_sum > max_sum
            max_sum = Math.max(current_sum, max_sum);
        }
    }
    
    return max_sum;
}

var arr = [1, 4, 2, 10, 23, 3, 1, 0, 20];
console.log(maxSum(arr, 4)); // 39

39


In [16]:
// using sliding window technique
// O(n)

function maxSumSW(arr, k) {
    if(arr.length < k) {
        return -1;
    }
    
    // get the sum of the first k elements
    let max_sum = 0;
    for(let i = 0; i < k; i++) {
        max_sum += arr[i];
    }
    
    let window_sum = max_sum;
    
    // then as we iterate from k --> n
    // we add the next value to our window
    // and remove the first one on our window
    // so if arr = [1, 4, 2, 10, 23, 3, 1, 0, 20]
    // and our current window = [1, 4, 2, 10]
    // then we add [23] to the window
    // and remove [1]
    // so new_window = [4, 2, 10, 23]
    for(let i = k; i < arr.length; i++) {
        window_sum += arr[i] - arr[i - k];
        
        max_sum = Math.max(window_sum, max_sum);
    }
    
    return max_sum;
}

var arr = [1, 4, 2, 10, 23, 3, 1, 0, 20];
console.log(maxSumSW(arr, 4)); // 39

39


<a class="anchor" id="linkedlists"></a>
# Linked Lists

### General Stuff:
* LL = data structure that stores items in a linear fashion where each element has a pointer that points to the next element in the list
    - singly LL: contains 1 pointer that points to the next element in the list only
    - doubly LL: contains 2 pointers that points to the next and previous elements in the list
* why use a LL?
    - saves memory by only allocating what's neded for elements
    - since each node has a pointer to the next/previous element in the list, a linked list doesn't need to occupy contiguous space in memory
* recursive definition: a linked list is either:
    - null
    - or a node pointing to a linked list
* most solutions are based on the runner technique where you have one slow pointer and another fast pointer
    - slow pointer moves 1 at a time while fast pointer can move 2 at a time
        - this method is really helpful for identifying if you have a loop in a linked list. they will both eventually meet at the same node
    - or fast pointer is a couple of steps ahead of the slow pointer but they both move at the same pace
    - regardless, the solution will be easier if you use 2 pointers to traverse the node quicker
* take into consideration the type of linked list you're dealing with
    - some things are easier than others to accomplish if you have a doubly linked list rather than a singly linked list
* recursive solutions are more elegant and concise but they always require at least O(n) space
* every problem requires some sort of traversal for linked lists
    - see if you can find the solution within 1 traversal
    - if not, see what kind of ways you can frontload any work to make traversal simpler

### Things to Ask:
1. what type of linked list are we working with
    - should be the FIRST question you ask b/c it could potentially simplify the question

### [Go back to top](#top)

In [31]:
// implementation of a doubly linked list
// with a head and tail pointer

class Node {
    constructor(data) {
        this.data = data;
        this.next = null;
        this.previous = null;
    }
}

// similates private properties
const head = Symbol('head');
const tail = Symbol('tail');

class DoublyLinkedList {
    constructor() {
        this[head] = null;
        this[tail] = null;
    }
    
    get(index) {
        if(index > -1) {
            let current = this[head];
            let i = 0;
            while(current !== null && i < index) {
                current = current.next;
                i++;
            }
            
            return current !== null ? current.data : undefined;
        }
        else {
            return undefined;
        }
    }
    
    push(data) {
        const newNode = new Node(data);
        
        if(this[head] === null) {
            this[head] = newNode;
        }
        else {
            this[tail].next = newNode;
            newNode.previous = this[tail];
        }
        
        this[tail] = newNode;
    }
    
    unshift(data) {
        const newNode = new Node(data);
        
        if(this[head] === null) {
            this[head] = newNode;
            this[tail] = newNode;
        }
        else {
            newNode.next = this[head];
            this[head] = newNode;
        }
    }
    
    remove(index) {
        if( (this[head] === null) || (index < 0)) {
            throw new RangeError('Index ${index} does not exist in the list.');
        }
        
        if(index === 0) {
            const data = this[head].data;
            
            this[head] = this[head].next;
            
            if(this[head] === null) {
                this[tail] === null;
            }
            else {
                this[head].previus = null;
            }
            
            return data;
        }
        
        let current = this[head];
        let i = 0;
        
        while( (current !== null) && (i < index)) {
            current = current.next;
            i++;
        }
        
        if(current !== null) {
            current.previous.next = current.next;
            
            if(this[tail] === current) {
                this[tail] = current.previous;
            }
            else {
                current.next.previous = current.previous
            }
            
            return current.data;
        }
        
        throw new RangeError(`Index ${index} does not exist in the list`);
    }
    
    *reverse() {
        let current = this[tail];
        while(current !== null) {
            yield current.data;
            current = current.previous;
        }
    }
    
    *values() {
        let current = this[head];
        while(current !== null) {
            yield current.data;
            current = current.next;
        }
    }
    
    [Symbol.iterator]() {
        return this.values();
    }
}

In [ ]:
// general recursive algorithm form for LL

function recAlgorithm(node) {
    if(node === null) {
        // base case
        // do something simple here
    }
    else {
        // recursive case
        // do smething at head of list, then call method on the
        // rest of the list
        recAlgorithm(node.next);
    }
}

In [ ]:
// recursve list reconstruction
// simple way to reconstruct a list
// able to do lots of non-trivial things with it

function construct(node) {
    if(node === null) {
        return null;
    }
    else {
        node.next = construct(node.next);
        return node;
    }
}

In [ ]:
// reversing a linked list


class LinkedListNode {
    constructor(data) {
        this.data = data;
        this.next = null;
    }
}

//only need to use 3 pointers
// current = current node being looked at
// previous = the one before current
// following = keeps track of next node in list

function reverseList(node) {
    let current = node;
    let previous = null;
    let following = null;
    
    while(current !== null) {
        following = current.next;
        current.next = previous;
        previous = current;
        current = following;
    }
}

## 2.1 Remove Dups
* remove duplicates from an unsorted linked list
* FOLLOW UP: how would you solve this problem if a temp buffer is not allowed?

In [47]:
// assume use of a singly linked list

class Node {
    constructor(data) {
        this.data = data;
        this.next = null;
    }
}

const head = Symbol('head');

class SinglyLinkedList {
    constructor() {
        this[head] = null;
    }
    
    add(data) {
        const newNode = new Node(data);
        
        if(this[head] === null) {
            this[head] = newNode;
        }
        else {
            newNode.next = this[head];
            this[head] = newNode;
        }
    }
    
    get(index) {
        if(index > -1) {
            if(index === 0) {
                return this[head];
            }
            let current = this[head];
            let i = 0;
            
            while(current !== null && i < index) {
                current = current.next;
                i++;
            }
            
            return current !== null ? current : undefined;
        }
        else {
            throw new RangeError(`The index ${index} is not in range!`)
        }
    }
    
    *values() {
        let current = this[head];
        while(current !== null) {
            yield current.data;
            current = current.next;
        }
    }
    
    [Symbol.iterator]() {
        return this.values();
    }
}

In [48]:
// O(n) time
// O(n) space b/c of hash table

function removeDups(node) {
    let hash = {};
    let previous = null;
    while(node !== null) {
        let data = node.data;
        
        // if we've seen it already
        // then remove it by setting previous.next = node.next
        // which makes previous.next pointer skip over node
        if(hash[data]) {
            previous.next = node.next;
        }
        // else add it to the hash table
        else {
            hash[data] = data;
            previous = node;
        }
        node = node.next;
    }
}

var s = new SinglyLinkedList();
s.add(1);
s.add(2);
s.add(2);
s.add(3);
s.add(3);
s.add(3);
console.log([...s]);
removeDups(s.get(0));
console.log([...s])

[ 3, 3, 3, 2, 2, 1 ]
[ 3, 2, 1 ]


In [50]:
// O(n^2) time b/c it's essentially a nested for-loop
// O(1) space

function removeDups2(node) {
    let current = node;
    
    while(current !== null) {
        let runner = current;
        while(runner.next !== null) {
            if(runner.next.data === current.data) {
                runner.next = runner.next.next;
            }
            else {
                runner = runner.next;
            }
        }
        current = current.next;
    }
}

var s = new SinglyLinkedList();
s.add(1);
s.add(2);
s.add(2);
s.add(3);
s.add(3);
s.add(3);
console.log([...s]);
removeDups2(s.get(0));
console.log([...s])

[ 3, 3, 3, 2, 2, 1 ]
[ 3, 2, 1 ]


## Loop Detection
* Given a circular linked list, implement an algorithm that returns the node at the beginning of the loop.
* Definition: A (corrupt) linked list in which a node's next pointer points to an earlier node, so as to make a loop in the linked list.
* Example:
Input: A -> B -> C -> D -> E -> C (the same C as earlier)
Output: C

In [ ]:
// use of the runner technique for linked lists
// you have one pointer moving slower
// and another one moving faster

function findBeginning(head) {
    let slow = head;
    let fast = head;
    
    // finds a point where slow and fast meet
    // slow moves 1 node at a time
    // fast moves 2 nodes at a timer
    while(fast !== null && fast.next !== null) {
        slow = slow.next;
        fast = fast.next.next;
        // they both meet
        if(slow === fast) {
            break;
        }
    }
    
    // error check. if fast is null, then it has no loop
    if(fast === null || fast.next === null) {
        return null;
    }
    
    // move slow to head. keep fast at meeting point. 
    // each are k steps from the loop start.
    // if they move at the sam pace, they must meet at loop start
    slow = head;
    while(slow !== fast) {
        slow = slow.next;
        fast = fast.next;
    }
    
    return fast;
}

<a class="anchor" id="stacks"></a>
# Stacks and Queues

### General Stuff:
* Stacks: 
    - LIFO Ordering Principle: last in, first out.
        - last element inserted into the stack is the first one out
    - really important for when you want to reverse the order of things
    - recursive algorithms implicitly use a stack
* Queues:
    - FIFO Ordering Principle: first in, first out
        - first element inserted into the queue is the first element out
    - helps with implementing breadth first search
* use stacks to reverse things and use queues to do things in order
* a queue can be made by two stacks
    - one stack is the inbox that takes all enqueues
    - pop all contents of inbox and push onto outbox to get FIFO order
* remember that these are abstract data types so how these things are implemented can be important for the question
    - e.g. you can store extra info like in the min problem or you can keep time data like in the animal shelter problem
    
### Things to Ask:
1. Are these stacks and queues standard or can I modify their implementation somehow to fit the problem?

### [Go back to top](#top)

In [1]:
// Stack implementation

class Stack {
    constructor() {
        this.items = [];
        this.size = 0;
    }
    
    // inserts item to top of the stack
    push(item) {
        this.items.push(item);
        this.size++;
    }
    
    // removes item from top of stack and returns it
    pop(item) {
        this.size--;
        return this.items.pop();
    }
    
    // returns the top item from the stack w/o removal
    peek() {
        return this.items[this.size - 1];
    }
    
    isEmpty() {
        return this.size === 0;
    }
}

In [2]:
// Queue Implementation

// enqueue = O(1)
// dequeue = O(n)
class Queue {
    constructor() {
        this.items = [];
        this.size = 0;
    }
    
    enqueue(item) {
        this.items.push(item);
        this.size++;
    }
    
    dequeue(item) {
        this.items.shift();
        this.size--;
    }
}

// enqueue = O(n)
// dequeue = O(1)
class Queue {
    constructor() {
        this.items = [];
        this.size = 0;
    }
    
    enqueue(item) {
        this.items.unshift(item);
        this.size++;
    }
    
    dequeue() {
        this.size--;
        return this.items.pop();
    }
}

// using a doubly linked list with head/tail pointers
// enqueue = O(1)
// dequeue = O(1)

class node {
    constructor(data) {
        this.data = data;
        this.next = null;
        this.previous = null;
    }
}

const head = Symbol('head');
const tail = Symbol('tail');

class Queue3 {
    constructor() {
        this[head] = null;
        this[tail] = null;
        this.size = 0;
    }
    
    enqueue(item) {
        const newNode = new node(item);
        if(this[head] === null) {
            this[head] = newNode;
            this[tail] = newNode;
        }
        else {
            newNode.next = this[head];
            this[head].previous = newNode;
            this[head] = newNode;
        }
        this.size++;
    }
    
    dequeue() {
        // if list is not empty
        if(this[head] !== null) {
            let data;
            // list has 1 item
            if(this[head].next === null) {
                data = this[head].data;
                this[head] = null;
                this[tail] = null;
            }
            // list has more than 1 item
            // remove from tail
            else {
                data = this[tail].data;
                this[tail].previous.next = null;
                this[tail] = this[tail].previous;
            }
            this.size--;
            return data;
        }
        else {
            return undefined;
        }
    }
    
    isEmpty() {
        return this[head] === null;
    }
    
    get length() {
        return this.size;
    }
    
    *values() {
        let current = this[head];
        while(current !== null) {
            yield current.data;
            current = current.next;
        }
    }
    
    [Symbol.iterator]() {
        return this.values();
    }
}

## 3.1 Three in One
* describe how you could use a single array to implement three stacks

In [ ]:
// done by dividing the array into 3 equal parts

class ThreeInOne {
    constructor() {
        this.container = [];
        this.middleBottom = 0;
        this.middleTop = 0;
    }
    
    push1(item) {
        this.container.unshift(item);
        this.middleBottom++;
        this.middleTop++;
    }
    
    push2(item) {
        this.container.splice(this.middleTop, 0, item);
        this.middleTop++;
    }
    
    push3(item) {
        this.container.push(value);
    }
    
    pop1() {
        if(this.isEmpty1()) {
            return undefined;
        }
        let answer = this.container.shift();
        if(this.middleBottom > 0) {
            this.middleBottom--;
            this.middleTop--;
        }
        return answer;
    }
    
    pop2() {
        if(this.isEmpty2()) {
            return undefined;
        }
        
        let answer = this.container[this.middleTop - 1];
        this.container.splice(this.middleTop - 1, 1);
        if(this.middleBottom < this.middleTop) {
            this.middleTop--;
        }
        
        return answer;
    }
    
    pop3() {
        if(this.isEmpty3()) {
            return undefined;
        }
        return this.container.pop(value);
    }
    
    peek1() {
        return this.isEmpty1() ? undefined : this.container[0];
    }
    
    peek2() {
        return this.isEmpty2() ? undefined : this.container[this.middleTop - 1];
    }
    
    peek3() {
        return this.isEmpty3() ? undefined : this.container[this.container.length - 1];
    }
    
    isEmpty1() {
        return this.middleBottom === 0;
    }
    
    isEmpty2() {
        return this.middleBottom === this.middleTop;
    }
    
    isEmpty3() {
       return this.middleTop === this.container.length; 
    }
}

## 3.2 Stack Min
* how would you design a stack which, in addition to push and pop, has a function min which returns the minimum element? Push, pop, and min should all operate in O(1) time

In [ ]:
// can achieve this by using 2 stacks
// stack = keeps track of all inserted items
// minStack = keeps track of all min values every insertion

class StackMin {
    constructor() {
        this.stack = new Stack();
        this.minStack = new Stack();
        this.currMin = undefined;
    }
    
    // if we have no currMin or the item being pushed is <= currMin
    // then push the item onto our minStack
    // and currMin becomes the new value
    push(item) {
    push(item) {
        if(this.currMin === undefined || item.data <= this.currMin) {
            this.minStack.push(this.currMin);
            this.currMin = value;
        }
        this.stack.push(value);
    }
    
    // if the popped element is actually the current min
    // then pop it off minStack
    // and set currMin = top of minStack (new currMin)
    pop() {
        let answer = this.stack.pop();
        if(answer === this.currMin) {
            this.currMin = this.minStack.peek();
        }
        return answer;
    }
    
    peek() {
        return this.stack.peek();
    }
    
    min() {
        return this.currMin;
    }
}

## 3.3 Stack of Plates
* Imagine a (literal) stack of plates. If the stack gets too high, it might topple. Therefore, in real life, we would likely start a new stack when the previous stack exceeds some threshold. Implement a data structure SetOfStacks that mimics this. SetOfStacks should be composed of several stacks and should create a new stack once the previous one exceeds capacity. SetOfStacks.push() and SetOfStacks.pop() should behave identically to a single stack (that is, pop() should return the same values as it would if there were just a single stack).
* FOLLOW UP:
    - Implement a function popAt(int index) which performs a pop operation of a specific sub-stack.

In [ ]:
// use an array of stacks
// as we reach capacity for one stack, we then make a new one
// and push onto that one

// follow-up:
// take in index and use it to find out which stack in the array
// so arr[index] = stack we want to pop from

class SetofStacks {
    constructor() {
        const stackOne = new Stack();
        this.set = [stackOne];
        // keeps track of the currentStack in use
        this.currentStack = 0;
    }
    
    addStack(item) {
        const newStack = new Stack();
        newStack.push(item);
        this.set.push(newStack);
        this.currentStack++;
    }
    
    push(item) {
        const {set, currentStack} = this;
        // if current stack exceeded capacity
        if(set[currentStack].length >= 3) {
            this.addStack(item);
        }
        // else if there is still room
        else {
            set[currentStack].push(item);
        }
    }
    
    pop() {
        const {set, currentStack} = this;
        const popped = set[currentStack].pop();
        if(set[currentStack].length === 0) {
            this.currentStack--;
        }
        return popped;
    }
    
    // follow-up:
    popAt(index) {
        if(index > -1 && index <= this.currentStack) {
            return this.set[index].pop();
        }
        else {
            return undefined;
        }
    }
}

## 3.4 Queue via Stacks
* Implement a MyQueue class which implements a queue using two stacks

In [3]:
// inbox = all items are enqueued into this
// outbox = all items are popped from this
// if outbox = empty, then pop all elements from inbox
// and push them to the outbox

// essentially, since the most recent item in the inbox is the first
// element pushed onto the outbox, 
// it becomes the oldest element in outbox
// thus, oldest element in the inbox becomes the first one to be popped
// in the outbox
// which maintains the FIFO ordering in queues

const inbox = Symbol('inbox');
const outbox = Symbol('outbox');

class MyQueue {
    constructor() {
        this[inbox] = new Stack();
        this[outbox] = new Stack();
        this.size = 0;
    }
    
    enqueue(item) {
        this[inbox].push(item);
        this.size++;
    }
    
    dequeue() {
        if(this[outbox].isEmpty()) {
            while(!this[inbox].isEmpty()) {
                this[outbox].push(this[inbox].pop());
            }
        }
        this.size--;
        return this[outbox].pop();
    }
    
    isEmpty() {
        return this.size === 0;
    }
}

## 3.6 Animal Shelter
* An animal shelter, which holds only dogs and cats, operatres on a strictly "first in, first out" basis. People must adopt either the "oldest" (based on arrival time) of all animals at the shelter, or they can select whether they would prefer a dog or a cat (and will receive the oldest animal of that type). They cannot select which specific nimal they would like. Create the data structures to maintain this system, and implement operations such as enqueue, dequeueAny, dequeueDog, and dequeueCat. You may use the built-in LinkedList data structure.

In [ ]:
// have separate queues for dogs and cats and for all animals
// we enqueue by type

class AnimalShelter {
    constructor() {
        this.dogQ = new Queue();
        this.catQ = new Queue();
        this.allQ = new Queue();
        this.tempQ = new Queue();
    }
    
    enqueue(animal) {
        if(animal.type === 'dog') {
            this.dogQ.enqueue(animal);
        }
        else if (animal.type === 'cat'){
            this.catQ.enqueue(animal);
        }
        this.allQ.enqueue(animal);
    }
    
    // check which animal is the oldest and dequeue it from Any
    // then dequeue the from the queue that houses its type
    dequeueAny() {
        if(this.allQ.peek() === this.dogQ.peek()) {
            this.dogQ.dequeue();
        }
        else if (this.allQ.peek() === this.catQ.peek()) {
            this.catQ.dequeue();
        }
        return this.allQ.dequeue();
    }
    
    // checks what animal type it is
    // dequeues from allQ until it finds the right type
    // and all those dequeues are added to a temporary queue
    // once the animal type is dequeued properly
    // we enqueue all of allQ into tempQ so that it recreates its order
    // then we enqueue all of tempQ back into allQ
    // it's done like this b/c we can't splice the type out
    dequeueByType(type) {
        let yesQ = type === 'dog' ? this.dogQ : this.catQ;
        
        while(!this.allQ.isEmpty() && this.allQ.peek().type !== type) {
            this.tempQ.enqueue(this.allQ.dequeue());
        }
        
        let animal = this.allQ.dequeue();
        yesQ.dequeue();
        
        while(!this.allQ.isEmpty()) {
            this.tempQ.enqueue(this.allQ.dequeue());
        }
        
        while(!this.tempQ.isEmpty()) {
            this.allQ.enqueue(this.tempQ.dequeue());
        }
        
        return animal;
    }
    
    dequeueDog() {
        return this.dequeueByType('dog');
    }
    
    dequeueCat() {
        return this.dequeueByType('cat');
    }
}

<a class="anchor" id="trees"></a>
# Trees and Graphs

### General Stuff:
* Trees:
    - tree is a set of nodes and a set of edges that connect pairs of nodes
    - has a root node and every node has a parent except the root
    - each node has a unique path from root --> itself
    - binary tree = each node has a max of 2 children
    - recursive definition: 
        - base case: a tree is either empty or
        - recursive case: consists of a root and zero or more subtrees, each of which is also a tree
    - level = level of a node n is the number of edges on that path from the root to n
    - height = max level of any node in the tree. so if we have a root node that has 2 children, max level and thus height would be 1, because there is 1 edge between root and its children
* Priority Queues with Binary Heaps:
    - similar to queue but front of priority queue contains high priority items and lower priority items are in the back
    - can be implemented using a binary heap which is like a binary tree
        - binary heap allows for O(logn) enqueue and dequeue
    - can implement a heap using an array
    - common variations of a binary heap:
        - min heap: smallest key at front
        - max heap: largest key at front
    - to ensure logarithmic performance on operations, tree will have to be balanced by creating a complete binary tree
        - complete meaning that each level of the tree is filled in except for the rightmost position in the bottom level
    - can represent the heap using a single list
        - parent = index p;
        - left child = index 2p;
        - right child = index 2p + 1;
        - and to find a parent of a node, it is Math.floor(n / 2)
    - heap order property:
        - for every node x with parent p, p.key <= x.key for min heaps
        - for max heaps: p.key >= x.key
    - heap operations:
        - insert:
            - insert new node at rear of the heap (bottom of binary tree)
            - use a helper function to move the item up the tree until it is in the right place
            - essentially, the new node will compare its value with its parent's value, then if new < parent, then swap places between parent and new (for min)
        - delMin:
            - put value of root into a variable
            - put last item in heap into the root
            - move this item down the tree, swapping with items that are smaller than it
            - then return the min value
    - build heap from an array in O(n) operations
        - starting from middle of array and moving backwards, we essentially percolate items down
        - starting from the middle ensures the largest item is moved down the entire tree
        - b/c the heap is a complete binary tree, any nodes past the halfway point of the array will be leaves
        - the reason why it is O(n) is b/c log n is derived from the height of the tree, and the tree is actually shorter than log n for most of the work in building a heap from the array
        - this is useful for sorting using a heap in O(nlogn) time
* Binary Search Trees
    - operations:
        - put = inserts a new item into the bst
            - limiting factor = height of the tree
            - worst case of a balanced binary tree is O(log n) b/c half of the elements will be less than the root and the other half will be greater than the root
            - if the keys were inserted in sorted order, then we will have the tree be skewed in one direction and thus it will be O(n) since the tree will have more trees in either the left or right subtree by a substantial amount
        - del = deletes an item in the bst
            - also O(log n) b/c it has to find the successor which is about O(log n) as well
        - get = returns the value at a node with matching key. accepts key as param
            - also O(log n) b/c it has to traverse the tree to find the right one
    - BST Property:
        - key values less than its parent are in the left subtree
        - key values greater than its parent are in the right subtree
        - this property applies for every node in the tree
    - three cases to consider when we want to delete a node containing the key we want deleted
        1. node to be deleted as no children:
            - easiest b/c it is leaf
            - just remve any reference of it from the parent
        2. node to be deleted has only 1 child
            - replace current node with its child
        3. node to be deleted as 2 children
            - go into the current node's right subtree and find its successr
            - basically, we find the min element in the right subtree so that we find the next largest item after our current node
            - if the successor is literally the right child of our current node, we can just replace our current node with it
            - if the successor is not the right child, we replace successor by its own right child, then we replace the current node with the successor to maintain bst property
    - on average, all operations are O(log n) where height of bst = log n
        - if keys are inserted into a binary tree in SORTED ORDER, then we will be skewing the tree in either the left or right subtrees substantially, and thus performance will be reduced to O(n)
* Balanced Binary Search Trees:
    - easiest way to create a balanced binary search tree given an array of keys is to:
        1. sort array of keys. this will be O(n log n) if you use something like merge sort
        2. then you first insert the middle element in the array to act as the root
            - since it is already sorted, the middle element, will be able to split the list of elements in half on either side
        3. then you split the arrays into left and right subarrays and find the middle of those to insert into the tree
        4. you can do the left and right splits recursively by inserting from the middle in the left subarray first, then move onto the middle of the right subarray
    - if we inserted the keys randomly, we cannot guarantee that the root will be able to split the list of elements evenly and thus operations might degrade to O(n)
    - however, there is a variation of the bst thatallows for balancing on every insert
        - this is called an AVL TREE
* AVL Tree:
    - keeps track of a balance factor of each node by looking at the heights of the left and right subtrees of each node
        - balanceFactor(bf) = height(leftSubTree) - height(rightSubTree)
        - if bf = -1, 0, 1, then the tree is balanced
        - if bf > 1, tree is left heavy, meaning there are more nodes in the left subtree than the right
        - if bf < -1, then the tree is right-heavy, meaning there are more nodes in the right subtree than the left
    - searching in our AVL tree will ensure O(logn) performance b/c at any time, the height of the tree is equal to 1.44 * log N where N = number of nodes in the tree
    - implementation details:
        - if new node is a left child, increase bf by 1
        - if new node is a right child, decrease bf by 1
        - updating bf can be done recursively. the base cases:
            - recurisve call reached root of tree
            - bf of parent has been adjusted to zero. once the parent has a bf of zero, then we can assume that the balance of its ancestor nodes does not change
    - how do we actually rebalance a tree?
        - we can perform rotations on it
        - left rotation:
            1. promote right child to be root of subtree
            2. move old root to be left child of new root
            3. if new root had a left child, make it the right child of the new left child
        - right rotation:
            1. promote left child to be root of subtree
            2. move old root to be right child of new root
            3. if new root already had a right child, make it the left child of the new right child
    - what is the time complexity of our put (insert) method now that we have to rebalance the tree?
        - updating balance factors of all parents is O(log n), one for each level of the tree
        - if a subtree is out of balance, we just do 2 rotations and each rotation works in O(1) time
        - so in total, the put operation is still O(log n)
* Graphs:
    - graph G can be represented by G = (V, E) where V is a set of vertices and E is a set of edges
        - each edge is a tuple (v,w) where v,w both belong to the set of vertices
            - can also add a weight to the edges as well so E = (v, w, c) where c is the weight
    - path: sequences of vertices that are connected by edges from one vertex to another
    - cycle: cycle in a directed graph is a path that starts and ends at the same vertex
    - main ways to implement a graph:
        - adjacency matrix
        - adjacency list
* adjacency matrix:
    - 2D matrix to represent a graph
    - value stored at row v and column w represent an edge between them, meaning those 2 nodes are adjacent to each other
    - advantage:
        - simple
        - easy to see node connections
        - good to use when the numbers of edges in a graph are large
    - disadvantage:
        - not great for 'sparse' data meaning that the data does not fill out the grid entirely so there's a lot of empty space
        - would essentially need a graph that has V^2 edges to be efficient
* adjacency list:
    - has a list of all vertices in the graph
        - each vertex also has a list of other vertices that it is connected to
    - our implementation will use a hash table rather than an array
    - advantage:
        - compactly represents sparse data
        - able to look at a vertex and know about every other node it is connected to. unlike an adjacency matrix, you don't iterate thrugh the entire matrix to find that info out
* Breadth First Search (BFS)
    - breadth first search finds all nodes 1 level at a time starting from some node s
        - so given a node s, BFS will find all adjacent nodes to s, then for each of these adjacent nodes, it will find adjacent nodes to those adjacent nodes
    - bfs colors each vertex white, gray, or black
        - every node is initially white
        - if it is discovered, it becomes gray
        - when every node adjacent to that node is discovered, then it becomes black so no white ndes adjacent to it. only gray or black ones
    - BFS uses a Queue to determine which node to discover next
        - remember bbq
        - b = breadth first search and q = queue
    - Analysis:
        - the while loop iterates through every node in the graph once so it is O(V)
        - the for loop iterates through every edge of that particular vertex and we only do it for a node that has been dequeued, therefore it is O(E)
        - total = O(V + E)
* Depth First Search (DFS)
    - goal is to search as deeply as possible starting from one node, then moving onto subsequent nodes
    - uses recursion to do the job so it implicitly uses a stack
    - it essentially creates a tree or maybe even several trees
        - several trees = depth first forest
    - uses 2 additional instance variabes: discovery and finis htimes
        - discovery: tracks number of steps in algorithm before a vertex is first encountered
        - finish: number of steps in algorithm before a vertex is colored black
    - parenthesis property: all children of a particular node in a depth first tree have later discovery times and earlier finish times than their parents
        - for example, the deepest node in a branch is the first one to finish being explored and will have a lower finish time than its parent who will be explored last
    - Analysis:
        - both loops in dfs run in O(V) time b/c they iterate over all vertices in the graph
        - the loop in dfsvisit is executed once for each edge in the adjacency list of the current vertex and is only called if the vertex is white. it will only execute a max of once for every edge in the graph so it is O(E)
        - total = O(V + E)
* Topological Sort:
    - takes a directed acyclic graph and sorts it by vertices
    - so if you have a graph with an edge (v,w), then sort would have the ordering be v --> w
    - good if you have a graph with multiple steps and you want to find the right order of steps or for making schedules
    - algorithm:
        1. call dfs on a graph to compute finish times
        2. stores each vertex in a list in decreasing order of finish time
        3. return ordered list
* Strongly Connected Components:
    - scc: part of the graph where every pair of vertices (v,w) has a path from v --> w and w --> v
    - it is essentially a subgraph within a larger graph that has 1 node that connects to another scc
    - we will b making use of the reversed graph, Gr
        - this is where all the edges are reversed
        - so if there is an edge from v --> w, then in Gr, the edge is now from w --> v
    - Algorithm:
        1. call dfs for the graph G to compute finish times for each vertex
        2. compute Gr
        3. call dfs for Gr but in the main loop of DFS, explore each vertex in decreasing order of finish time
        4. each tree in the forest computed in step 3 is an scc. just print the ids for each vertex in each tree in the forest
    - create a transposition of the graph
        1. iterate through the vertList of the graph you want to transpose
        2. for each vertex, iterate through its adjacency list
        3. for each neighbor in the adjacency list, add an edge from it the the currentVertex
    - Analysis:
        - initial dfs over graph G is O(V + E)
        - computing transposition of graph G to make Gr is also O(V + E)
        - and calling dfs for graph Gr is also O(V + E)
        - total is about O(V + E) to find scc of a graph
* Dijkstra's Algorithm:
    - finds the shortest path from one starting node to every other node in the graph
    - similar to bfs
    - iterates through every vertex once in the graph but the iteration is based on a priority queue
        - uses a min-heap that is sorted by distance
        - so the node with the smallest distance from the starting node will be the first to be removed from the queue
    - implementation without decreaseKey method in BinaryHeap
        1. set startingnode.distance = 0, and every other node = Number.MAX_VALUE
        2. intialize a min priority queue and add the starting node in
        3. delete the minimmum (first item in min pq) and iterate through its adjacent vertices
        4. if the neighbor's current distance > currentVertex.distance + the weight between currentVertex and neighbor, then set neighbor's distance to be currentVertex.distance + weight of edge between them
        5. also set the predecessor of that neighbor to be currentVertex
        6. and add that neighbor to the priority queue where it might bubble to the top if it has a low enough distance
        7. keep repeating these steps until the priority queue is empty
    - Analysis:
        - deleting the minimum takes O(log V) b/c have to restore the heap order
        - it will iterate through every vertex and also every edge as well so it is O(V + E)
        - total = O(V + E) * O(log V) = O((V + E)logV)
* Prim's Spanning Tree Algorithm
    - this algorithm solves problems where you want to reach all other locations from one location without having to travel too far
    - so given a router, i want my internet connection to be able to reach all people in my house efficiently
    - minimum weight spanning tree:
        - a tree T for a graph G = (V, E) where T is acyclic
        - and all edges connects all vertices in V
        - such that the total weight between all edges is minimized
    - implementation: pretty similar to dijkstra's b/c uses priority queue
* be very comfortable with using inorder, preorder, and postorder traversals for trees as well as dfs and bfs for graphs
    - most algorithms for these problems are basically modified versions of them and can make your life way easier
    - know exactly when to use them
* be very comfortable with using recursion b/c almost every single traversal is recursive
    - understand what is going on with the variables in each recursive call
    - do they get passed to each recursive call
    - do the variables change their value if they get modified in another recursive call?
    
### Things to Ask:
1. What kind of tree/graph are we working with?
    - can we assume that the tree is a binary search tree or a regular binary tree?
    - can we assume that we are using an adjacency list to represent a graph or is it an adjacency matrix?
### [Go back to top](#top)

In [1]:
// implementation using a binary tree class and treenode class

class TreeNode {
    constructor(value) {
        this.value = value;
        this.left = null;
        this.right = null;
    }
}

const root = Symbol('root');
class BinaryTree {
    constructor() {
        this[root] = null;
    }
    
    // insertLeft and insertRight both insert at the root
    // and push the rest of the left/right subtrees down
    // so the tree grows at the roots
    insertLeft(item) {
        const newNode = new TreeNode(item);
        // if tree is empty, just insert into tree
        if(this[root] === null) {
            this[root] = newNode;
        }
        // if tree has a left child, the root's left child is now
        // the newNode and left subtree is pushed down one 
        else if (this[root] !== null && this[root].left !== null) {
            newNode.left = this[root].left;
            this[root].left = newNode;
        }
        // else just insert at the left since this[root].left = null
        else {
            this[root].left = newNode;
        }
    }
    
    insertRight(item) {
        const newNode = new TreeNode(item)
        if(this[root] === null) {
            this[root] = newNode;
        }
        else if (this[root] !== null && this[root].right !== null) {
            newNode.right = this[root].right;
            this[root].right = newNode;
        }
        else {
            this[root].right = newNode;
        }
    }
}

In [ ]:
// tree traversals

// preOrder
// root --> left --> right
function preOrder(node) {
    if(node !== null) {
        console.log(node);
        preOrder(node.left);
        preOrder(node.right);
    }
}

// inOrder
// left --> root --> right
function inOrder(node) {
    if(node !== null) {
        inOrder(node.left);
        console.log(node);
        inOrder(node.right);
    }
}

// postOrder
// left --> right --> root
function postOrder(node) {
    if(node !== null) {
        postOrder(node.left);
        postOrder(node.right);
        console.log(node);
    }
}

In [2]:
// implementation of a min Binary Heap
// use of a single array to simulate a tree

class BinHeap {
    constructor() {
        // the zero in the array is not used
        // but is there so that simple int. division can be used
        // in later methods
        this.heapList [0];
        this.currentSize = 0;
    }
    
    insert(k) {
        // adds element to end of tree
        this.heapList.append(k);
        this.currentSize++;
        // then moves the element up the tree
        // into the proper position
        this.percUp(this.currentSize);
    }
    
    percUp(i) {
        // i = current position of our node
        // p = parent
        let p = Math.trunc(i / 2);
        while(p > 0) {
            // if newNodes value < parent's value, then swap them
            if(this.heapList[i] < this.heapList[p]) {
                [this.heapList[i], this.heapList[p]] = [this.heapList[p], this.heapList[i]];
            }
            i = p;
            p = Math.trunc(i / 2);
        }
    }
    
    delMin() {
        // at 1 b/c we have a 0 at the beginning
        const retVal = this.heapList[1];
        // put last node in bin heap to the root position
        this.heapList[1] = this.heapList[this.currentSize];
        this.currentSize--;
        this.heapList.pop();
        // then move the node down to restore the bin heap order
        this.percDown(1);
        return retVal;
    }
    
    percDown(i) {
        // while the current node still has children
        while( (i * 2) <= this.currentSize) {
            // get the smallest child of the two
            let mc = this.minChild(i);
            // if the current node is greater than its child
            // then swap the two
            if(this.heapList[i] > this.heapList[mc]) {
                [this.heapList[i], this.heapList[mc]] = [this.heapList[mc], this.heapList[i]];
            }
            i = mc;
        }
    }
    
    minChild(i) {
        let leftChild = i * 2;
        let rightChild = (i * 2) + 1;
        
        // if there is no right child, return left child
        if(rightChild > this.currentSize) {
            return leftChild;
        }
        // else if there is a right and left child
        // return the smallest of the 2 in value
        else {
            if(this.heapList[leftChild] < this.heapList[rightChild]) {
                return leftChild;
            }
            else {
                return rightChild;
            }
        }
    }
    
    // building a heap from an array
    buildHeap(alist) {
        let i = Math.trunc( alist.length / 2);
        this.currentSize = alist.length;
        this.heapList = [0] + alist;
        // starts from the middle and percolates elements down
        // until 0th index
        while(i > 0) {
            this.percDown(i);
            i--;
        }
    }
}

In [ ]:
class TreeNode {
    constructor(key, val, left = null, right = null, parent = null) {
        this.key = key;
        this.payload = val;
        this.left = left;
        this.right = right;
        this.parent = parent;
    }
    
    hasLeftChild() {
        return this.left !== null;
    }
    
    hasRightChild() {
        return this.right !== null;
    }
    
    isLeftChild() {
        return this.parent !== null && this.parent.right === this;
    }
    
    isRightChild() {
        return this.parent !== null && this.parent.right === this;
    }
    
    isRoot() {
        return this.parent === null;
    }
    
    isLeaf() {
        return !(this.hasAnyChild());
    }
    
    hasAnyChildren() {
        return this.hasLeftChild() || this.hasRightChild();
    }
    
    hasBothChildren() {
        return this.hasLeftChild() && this.hasRightChild();
    }
    
    replaceNodeData(key, value, left, right) {
        this.key = key;
        this.payload = value;
        this.left = left;
        this.right = right;
        if(this.hasLeftChild()) {
            this.left.parent = this;
        }
        if(this.hasRightChild()) {
            this.right.parent = this;
        }
    }
    
    findSuccessor() {
        let successor;
        // if current node has a right subtree, find the minimum of
        // the right subtree
        if(this.hasRightChild()) {
            successor = this.right.findMin();
        }
        // else if it does not have a right subtree, then it will have
        // an ancestor who is a left child of its parent
        // and this left child's parent is the successor
        // else if it is the right child, keep going up the tree
        else {
            if(this.parent !== null) {
                if(this.isLeftChild()) {
                    successor = this.parent;
                }
                else {
                    // we remove our node from the tree temporarily
                    // find the successor again
                    // then add it back
                    // this is b/c we call findSuccessor on the parent
                    // and since it has a rightChild(previous node), then
                    // it will assign the successor
                    // to the current node
                    // so we remove it temporarily until we find the correct node
                    this.parent.right = null;
                    successor = this.parent.findSuccessor();
                    this.parent.right = this;
                }
            }
        }
        return successor;
    }
    
    // the minimum in a binary search tree is the leftmost node in the
    // tree
    findMin() {
        let current = this;
        while(current.hasLeftChild()) {
            current = current.left;
        }
        return current;
    }
    
    spliceOut() {
        // if the node is a leaf, then change the parent's child
        // reference to be null depending on whether it was a right
        // or left child
        if(this.isLeaf()) {
            if(this.isLeftChild()) {
                this.parent.left = null;
            }
            else {
                this.parent.right = null;
            }
        }
        else if (this.hasAnyChild()) {
            if(this.hasLeftChild()) {
                if(this.isLeftChild()) {
                    this.parent.left = this.left;
                }
                else {
                    this.parent.right = this.left;
                }
                this.left.parent = this.parent;
            }
            else {
                if(this.isLeftChild()) {
                    this.parent.left = this.right;
                }
                else {
                    this.parent.right = this.right;
                }
                this.right.parent = this.parent;
            }
        }
    }
}

const root = Symbol('root');

class BinarySearchTree {
    constructor() {
        this[root] = null;
        this.size = 0;
    }
    
    get length() {
        return this.size;
    }
    
    // inserting items int bst
    put(key, val) {
        if(this[root] !== null) {
            this._put(key, val, this[root]);
        }
        else {
            this[root] = new TreeNode(key, val);
        }
        this.size++;
    }
    
    _put(key, val, currentNode) {
        if(key < currentNode.key) {
            if(currentNode.hasLeftChild()) {
                this._put(key, val, currentNode.left);
            }
            else {
                currentNode.left = new TreeNode(key, val, null, null, currentNode);
            }
        }
        else {
            if(currentNode.hasRightChild()) {
                this._put(key, val, currentNode.right);
            }
            else {
                currentNode.right = new TreeNode(key, val, null, null, currentNode);
            }
        }
    }
    
    get(key) {
        if(this[root] === null) {
            return undefined;
        }
        const result = this._get(key, this[root]);
        return result ? result.payload : undefined;
    }
    
    _get(key, currentNode) {
        if(currentNode === null) {
            return undefined;
        }
        else if (currentNode.key === key) {
            return currentNode;
        }
        else if (key < currentNode.key) {
            return this._get(key, currentNode.left);
        }
        else {
            return this._get(key, currentNode.right);
        }
    }
    
    del(key) {
        if(this.size > 1) {
            const nodeToRemove = this._get(key, this[root]);
            if(nodeToRemove) {
                this.remove(nodeToRemove);
                this.size--;
            }
            else {
                return undefined;
            }
        }
        else if (this.size === 1 && this[root].key === key) {
            this[root] = null;
            this.size--;
        }
        else {
            return undefined;
        }
    }
    
    remove(currentNode) {
        if(currentNode.isLeaf()) {
            if(currentNode.isLeftChild()) {
                currentNode.parent.left = null;
            }
            else {
                currentNode.parent.right = null;
            }
        }
        else if (currentNode.hasBothChildren()) {
            let successor = currentNode.findSuccessor();
            successor.spliceOut();
            currentNode.key = successor.key;
            currentNode.payload = successor.payload;
        }
        else {
            if(currentNode.hasleftChild()) {
                if(currentNode.isLeftChild()) {
                    currentNode.left.parent = currentNode.parent;
                    currentNode.parent.left = currentNode.left;
                }
                else if (currentNode.isRightChild()) {
                    currentNode.left.parent = currentNode.parent;
                    currentNode.right.parent = currentNode.left;
                }
                else {
                    currentNode.replaceNodeData(currentNode.left.key,
                                                currentNode.left.payload,
                                                currentNode.left.left,
                                                currentNode.left.right);
                }
            }
            else {
                if(currentNode.isleftChild()) {
                    currentNode.right.parent = currentNode.parent;
                    currentNode.parent.left = currentNode.right;
                }
                else if (currentNode.isRightChild()) {
                    currentNode.right.parent = currentNode.parent;
                    currentNode.parent.right = currentNode.right;
                }
                else {
                    currentNode.replaceNodeData(currentNode.right.key,
                                                currentNode.right.payload,
                                                currentNode.right.left,
                                                currentNode.right.right);
                }
            }
        }
    }
}

In [8]:
// adjacency list implementation

class Vertex {
    constructor(key) {
        this.id = key;
        this.connectedTo = new Map();
    }
    
    //adds a neighbor to this vertex
    // weight is default 0 if unweighted
    addNeighbor(nbr, weight = 0) {
        // key = nbr (Vertex object)
        // value = weight (primitive)
        this.connectedTo.set(nbr, weight);
    }
    
    // returns list of connections to this vertex via the keys
    // in the connectedTo object
    getConnections() {
        return this.connectedTo;
    }
    
    getId() {
        return this.id;
    }
    
    // returns the weight of an edge between this node and a neighbor
    getWeight(nbr) {
        return this.connectedTo.get(nbr);
    }
}

class Graph {
    constructor(){
        this.vertList = new Map();
        this.numVertices = 0;
    }
    
    addVertex(key) {
        this.numVertices++;
        const newVertex = new Vertex(key);
        this.vertList.set(key, newVertex);
        return newVertex;
    }
    
    getVertex(key) {
        return this.vertList.has(key) ? this.vertList.get(key) : undefined;
    }
    
    addEdge(from, to, weight = 0) {
        if( !this.vertList.has(from) ) {
            this.addVertex(from);
        }
        if( !this.vertList.has(to) ) {
            this.addVertex(to);
        }
        this.vertList.get(from).addNeighbor(this.vertList.get(to), weight);
    }
    
    getVertices() {
        return this.vertList.keys();
    }
    
    *values() {
        for(let [key, value] of this.vertList) {
            yield value;
        };
    }
    
    [Symbol.iterator]() {
        return this.values();
    }
}

In [ ]:
// bfs implementation
// assumes Vertex class has 3 new class fields:
// distance = distance from starting node
// predecessor = the node that is before it and has an edge to it
// color = the color of the node to determine whether it has been fully explored

function bfs(g, start) {
    // start at some node and set its distance to 0
    // and its predecessor as none
    start.distance = 0;
    start.predecessor = null;
    // add it to the queue
    let queue = new Queue();
    queue.enqueue(start);
    
    // while the queue is not empty
    while(queue.size > 0) {
        // dequeue from the node queue
        let currentVert = queue.dequeue();
        // then for all of its neighbors
        // if the nbr is undiscovered (white)
        // then color it gray
        // set its distance to currentVert's distanc + 1
        // and its predecessor to the current vert
        // then add this neighbor to the queue to check its adjacent
        // nodes later
        for(let [nbr, value] of currentVert.getConnections()) {
            if(nbr.color === 'white') {
                nbr.color = 'gray';
                nbr.distance = currentVert.distance + 1;
                nbr.pred = currentVert;
                queue.enqueue(nbr);
            }
        }
        // once we have discovered all of a node's neighbors, color
        // the currentvertex black
        // and then start the loop over again with a dequeued node
        currentVert.color = black;
    }
}

In [9]:
class DFSGraph extends Graph {
    constructor() {
        super();
        this.time = 0;
    }
    
    dfs() {
        // sets all vertices to white first
        for(let [key, vertex] of this.vertList) {
            vertex.color = 'white';
            vertex.pred = -1;
        }
        
        // if the vertex is totally unexplored
        // then explore it deeply
        for(let [key, vertex] of this.vertList) {
            if(vertex.color === 'white') {
                this.dfsvisit(vertex);
            }
        }
    }
    
    
    dfsvisit(startVertex) {
        // sets the vertex's color to gray
        // and adds discovery time
        startVertex.color = 'gray';
        this.time++;
        startVertex.discovery = this.time;
        // then for each of the vertex's neighbors, we want to explore
        // them deeply
        for(let [nextVertex, value] of startVertex.getConnections()) {
            // if the vertex is white
            // then call dfsvisit on it again
            // and set predecessor of nextVertex to startVertex
            if(nextVertex.color === 'white') {
                nextVertex.pred = startVertex;
                this.dfsvisit(nextVertex);
            }
        }
        // once every adjacent vertex of start is explored deeply
        // then color it black
        // and set its finish time
        startVertex.color = 'black';
        this.time++;
        startVertex.finish = this.time;
    }
}

In [ ]:
// Topoplogical Sort implementation

class DFSGraph extends Graph {
    constructor() {
        super();
        this.time = 0;
        this.topSortArray = [];
    }
    
    dfs() {
        // sets all vertices to white first
        for(let [key, vertex] of this.vertList) {
            vertex.color = 'white';
            vertex.pred = -1;
        }
        
        // if the vertex is totally unexplored
        // then explore it deeply
        for(let [key, vertex] of this.vertList) {
            if(vertex.color === 'white') {
                this.dfsvisit(vertex);
            }
        }
    }
    
    
    dfsvisit(startVertex) {
        startVertex.color = 'gray';
        this.time++;
        startVertex.discovery = this.time;
        for(let [nextVertex, value] of startVertex.getConnections()) {
            if(nextVertex.color === 'white') {
                nextVertex.pred = startVertex;
                this.dfsvisit(nextVertex);
            }
        }
        startVertex.color = 'black';
        this.time++;
        startVertex.finish = this.time;
        // once a vertex has been fully explored
        // add it to the topSort array b/c it has a finish time
        // we do unshift b/c the startVertex has the lowest finish time
        // and subsequent nodes will have greater finish times
        this.topSortArray.unshift(startVertex);
    }
    
    // once we run dfs
    // we can then return the array
    // which will have the vertices in order
    topSort() {
        this.dfs();
        return this.topSortArray;
    }
}

In [ ]:
// Strongly Connected Components algorithm

class DFSGraph extends Graph {
    constructor() {
        super();
        this.time = 0;
        this.topSortArray = [];
    }
    
        dfs() {
        for(let [key, vertex] of this.vertList) {
            vertex.color = 'white';
            vertex.pred = -1;
        }
        
        for(let [key, vertex] of this.vertList) {
            if(vertex.color === 'white') {
                this.dfsvisit(vertex);
            }
        }
    }
    
    dfsvisit(startVertex) {
        startVertex.color = 'gray';
        this.time++;
        startVertex.discovery = this.time;
        for(let [nextVertex, value] of startVertex.getConnections()) {
            if(nextVertex.color === 'white') {
                nextVertex.pred = startVertex;
                this.dfsvisit(nextVertex);
            }
        }
        startVertex.color = 'black';
        this.time++;
        startVertex.finish = this.time;
        this.topSortArray.unshift(startVertex);
    }
    
    reverse() {
        const RGraph = new DFSGraph();
        // key = num, value = object
        for(let [key, currentVertex] of this.vertList) {
            for(let [nbr, value] of currentVertex.getConnections()) {
                RGraph.addEdge(nbr.id, key);
            }
        }
        return RGraph;
    }
    
    dfsSCC(RGraph) {
        for(let [key,vertex] of RGraph.vertList) {
            vertex.color = 'white';
            vertex.prod = -1;
        }
        
        // different from regular dfs by exploring each
        // vertex in decreasing order of finish times
        this.topSortArray.forEach(vertex => {
            let RVertex = RGraph.getVertex(vertex.id);
            if(RVertex.color === 'white') {
                console.log({SCC: RVertex.id});
                this.dfsvisitSCC(RVertex);
            }
        })
    }
    
    dfsvisitSCC(startVertex) {
        startVertex.color = 'gray';
        for(let [nextVertex, value] of startVertex.getConnections()) {
            if(nextVertex.color === 'white') {
                console.log({components: nextVertex.id});
                nextVertex.pred = startVertex;
                this.dfsvisitSCC(nextVertex);
            }
        }
        startVertex.color = 'black';
        startVertex.finish = this.time;
    }
    
    scc() {
        this.dfs();
        const reverse = this.reverse();
        this.dfsSCC(reverse);
        return;
    }
}

In [ ]:
// Dijkstra's Algorithm

function dijkstra(aGraph, start) {
    // sets all vertex distances to max number
    for(let [key, vertex] of aGraph.vertList) {
        vertex.distance = Number.MAX_VALUE;
    }
    // then the start node is set to 0 for distance
    start.distance = 0;
    // add the start vertex to the min priority queue
    let pq = new BinHeap();
    pq.buildHeap([start]);
    
    // while the priority queue has elements
    // remove the node with the smallest distance
    // then for each of its neighboring nodes, see if
    // current.distance + weight < neighbor.distance
    // if it is, then set neighbor.distance to it
    // and the neighbor.distance for currentVert
    // then reinsert it into the priority queue
    // so once the shortest paths are found from starting node --> every other node
    // then the algorithm ends
    while(!pq.isEmpty()) {
        let currentVert = pq.delMin();
        // weight = weight between currentVert and its neighbors
        for(let [nextVert, weight] of currentVert.getConnections()) {
            let newDist = currentVert.distance + weight;
            if(newDist < nextVert.distance) {
                nextVert.distance = newDist;
                nextVert.pred = currentVert;
                pq.insert(nextVert);
            }
        }
    }
}

In [ ]:
// Prim's Spanning Tree

function prim(aGraph, start) {
    // sets every vertex to have a Number.MAX_VALUE distance
    for(let [key, vertex] of aGraph.vertList) {
        vertex.distance = Number.MAX_VALUE;
    }
    // then sets start distance to 0
    // then build an array with startVertex and all its neighbors
    // then we build a priority queue from this array
    start.distance = 0;
    let pq = new BinHeap();
    let list = [start];
    for(let [nextVert, nextWeight] of start.getConnections()) {
        list.concat(nextVert);
    }
    // this will automatically put all the vertices into min-heap
    // order with the startVertex at the root since its distance = 0
    pq.buildHeap(list);
    
    // then we delete the minimum from the pq
    // and check whether we visited the node before or if the
    // weight is less than the distance it has
    // if both those conditions are true
    // then we update the distance and predecessor
    // insert it into the queue
    // then set it as visited so it won't be added to the queue again
    while(!pq.isEmpty()) {
        let currentVert = pq.delMin();
        for(let [nextVert, nextWeight] of currentVert.getConnections()) {
            if( (!nextVert.visited) && nextWeight < nextVert.distance) {
                nextVert.pred = currentVert;
                nextVert.distance = nextWeight;
                pq.insert(nextVert);
                nextVert.visited = true;
            }
        }
    }
}

## 4.1 Route Between Nodes
* given a directed graph, design an algorithm to find out whether there is a route between two nodes

In [ ]:
// can be solved by any graph traversal algorithm such as dfs or bfs
// going to use bfs here

function bfs(start, end) {
    let q = new Queue();
    start.distance = 0;
    start.pred = null;
    q.enqueue(start);
    
    while(q.size > 0) {
        let currentVertex = q.dequeue();
        for(let [nbr, weight] of currentVertex.getConnections()) {
            if(nbr === end) {
                return true;
            }
            if(nbr.color === 'white') {
                nbr.color = 'gray';
                nbr.pred = currentVertex;
                nbr.distance = currentVertex.distance + 1;
                q.enqueue(nbr);
            }
        }
        currentVertex.color = 'black';
    }
    return false;
}

## 4.2 Minimal Tree
* given a sorted (increasing order) array with unique integer elements, write an algorithm to create a binary search tree with minimal height

In [ ]:
// mergesort-esque solution
// start the middle and split the element into two halves
// continually do this until you reach 1 element subarrays
// then build the tree up from this subarrays

function createMinimalBST(array) {
    return _createMinimalBST(array, 0, arr.length - 1);
}

function _createMinimalBST(arr, start, end) {
    if(end < start) { return null; }
    let mid = Math.trunc(start + ((end - start) / 2));
    let n = new TreeNode(arr[mid]);
    n.left = _createMinimalBST(arr, start, mid - 1);
    n.right = _createMinimalBST(arr, mid + 1, end);
    return n;
}

## 4.4 Check Balanced
* Implement a function to check if a binary tree is balanced. For the purposes of this question, a balanced tree is defined to be a tree such that the heights of the two subtrees of any node never differ by more than one.

In [ ]:
// can just modify postorder traversal
// it'll check the left --> right --> root
// so basically it'll check from the leaves up to the root
// to see if it is balanced

function isBalanced(start) {
    let balanceFactor = this.postOrder(start);
    return balanceFactor <= 1;
}

function postOrder(node) {
    if(node !== null) {
        let left = postOrder(node.left) + 1;
        let right = postOrder(node.right) + 1;
        // the difference between the two heights will determine how balanced
        // it is 
        return Math.abs(left - right);
    }
    // if the node is null
    // return -1;
    // this ensures that an empty tree will be considered balanced too
    // and that leaves have a bf of 0
    else {
        return -1;
    }
}

## 4.5 Validate BST
* implement a function to check if a binary tree is a binary search tree

In [ ]:
// start with a range of min = NULL, and max = NULL
// if we go left, min = null, max = root.payload
// if we go right, min = root.payload, max = null
// so as we go down, we essentially check the range
// time: O(n) b/c will have to check every node
// space: O(log n), will only ever have log n functions on the stack
function checkBST(n) {
    return _checkBST(n, null, null);
}

function _checkBST(n, min, max) {
    if(n == null) { return true; }
    
    // if the node's value falls outside of the current range, return false;
    if( (min !== null && n.payload <= min) || (max !== null && n.payload > max)) {
        return false;
    }
    
    // check left and right subtrees and narrow down the range as you go
    // left range = previous min and current node's data
    // right range = current node's data and previous max
    if(!_checkBST(n.left, min, n.payload) || !_checkBST(n.right, n.payload, max)) {
        return false;
    }
    
    return true;
}

## 4.6 Successor
* write an algorithm to find the 'next' node (i.e. in-order successor) of a given node in a binary search tree. You may assume that each node has a link to its parent

In [ ]:
// succ = min. of the node's right subtree
// else if the node has no right subtree, then
// succ = the parent of an ancestor who is a left Child

function findMin(node) {
    while(node.left !== null) {
        node = node.left;
    }
    return node;
}

function successor(node) {
    if(node === null) { return; }
    let succ;
    if(node.right !== null) {
        succ = findMin(node.right);
    }
    else {
        let x = node;
        while(x !== null && x.parent !== null) {
            if(x.isLeftChild()) {
                succ = x.parent;
                break;
            }
            x = x.parent;
        }
    }
    
    return succ ? succ.key : null;
}

## 4.7 Build Order
* you are given a list of projects and a list of dependencies (which is a list of pairs of projects, where the second project is dependent on the first project). All of a project's dependencies must be built before the project is. Find a build order that will allow the projects to be built. If there is no valid build order, return an error.

In [ ]:
// for anything that needs to be in order, use topological sort
// topological sort will perform a depth first search
// then assign start and finish times to each one
// then it will be placed into the topSort array in 
// descending order of finish times

class DFSGraph extends Graph {
    constructor() {
        super();
        this.time = 0;
        this.topSortArr = [];
    }
    
    topSort() {
        this.dfs();
        return this.topSortArr;
    }
    
    dfs() {
        // sets all nodes to be the color white, i.e. unexplored
        for(let [key, vertex] of this.vertList) {
            vertex.color = 'white';
        }
        // for every node, explore as deeply as possible if they are
        // unexplored
        for(let [key, vertex] of this.vertList) {
            if(vertex.color === 'white') {
                this.dfsvisit(vertex);
            }
        }
    }
    
    dfsvisit(startVertex) {
        startVertex.color = 'gray';
        this.time++;
        startVertex.disc = this.time;
        
        for(let [nbr, weight] of startVertex.getConnections()) {
            if(nbr.color === 'white') {
                nbr.color = 'gray';
                nbr.pred = startVertex;
                this.dfsvisit(nbr);
            }
        }
        
        startVertex.color = 'black';
        this.time++;
        startVertex.fin = this.time;
        this.topSortArr.unshift(startVertex);
    }
}

## 4.10 Check Subtree
* T1 and T2 are two very large binary trees, with T1 much bigger than T2. Create an algorithm to determine if T2 is a subtree of T1.

In [ ]:
// search through the larger tree, T1, for any instance of the root of T2
// once we have found a match, we then call another function that checks whether the
// 2 subtrees are identical by doing an in-order traversal

function containsTree(t1, t2) {
    // empty tree is always a subtree. also true for empty arrays and strings too
    if(t2 == null) { return true; }
    return subTree(t1, t2);
}

function subTree(r1, r2) {
    // big tree empty and subtree is still not found
    if(r1 == null) {
        return false;
    }
    // if we found a match, see if their subtrees match up
    else if (r1.payload === r2.payload && matchTree(r1, r2)) {
        return true;
    }
    // else looks through the left and right subtrees for the instance of r2
    return subTree(r1.left, r2) || subTree(r1.right, r2);
}

function matchTree(r1, r2) {
    // if everything has been checked in the subtree ,return true;
    if(r1 == null && r2 == null) {
        return true;
    }
    // if either of the trees are empty, it cannot match cause one of them
    // still has more nodes to go
    if(r1 == null || r2 == null) {
        return false;
    }
    // if there are different values at the same position
    //then they do not match
    else if (r1.payload !== r2.payload) {
        return false;
    }
    // if the current nodes match, then go ahead and check
    // its left and right subtrees
    else {
        // checks the left and rightsubtrees
        // for a match
        return matchTree(r1.left, r2.left) && matchTree(r1.right, r2.right);
    }
}

## 4.12 Paths with Sum
* you are given a binary tree in which each node contains an integer value (which might be positive or negative). Design an algorithm to count the number of paths that sum to a given value. The path does not need to start or end at the root or a leaf, but it must go downwards (traveling only from parent nodes to child nodes)

In [1]:
// visit each node and track runningSum by incrementing it with node.payload
// look up runningSum - targetSum in hash table and set up totalPaths = that value
// if runningSum == targetSum, increment totalPaths
// add runningSum to hash table (inc if it's already there)
// recurse left and right, counting number of paths with sum targetSum
// after recursing, decrement value of runningSum in hash table which reverses changes to hash table
// so that other nodes don't use it
// reason why we use the hash table is b/c it keeps track of the sums that we have seen previously that
// do not start at the root. so if the current node's value - targetsum = any sum we have seen on the
// path from the root to the current node, then we know that we have a path for the targetsum
// time complexity: O(n) b/c visit every nod in the tree doing O(1) work
// space complexity: O(log n) for a balanced tree but can bloat to O(n) for an unbalanced tree
function countPathWithSum(root, targetSum) {
    let sumTable = {};
    return _countPathsWithSum(root, targetSum, 0, sumTable);
}

function _countPathsWithSum(node, targetSum, runningSum, sumTable) {
    if(node == null) { return 0;}
    
    runningSum += node.payload;
    let sum = runningSum - targetSum;
    
    // totalPaths keeps track of any sums we have seen so far on the way
    // that does not start at the root
    let totalPaths = sumTable[sum] === undefined ? 0 : sumTable[sum];
    
    if(runningSum === targetSum) {
        totalPaths++;
    }
    
    // initializes or increments the current runningSum into the hash
    // table and adds it by 1
    incrementHashTable(sumTable, runningSum, 1);
    
    // traverses left and right to check if the targetSum is in
    totalPaths += _countPathsWithSum(node.left, targetSum, runningSum, sumTable);
    totalPaths += _countPathsWithSum(node.right, targetSum, runningSum, sumTable);
    
    // once we have traversed all paths and returned back, we remove the runningSum before so that
    // once we branch to other portions of the tree, that particular sum will no longer be used
    // this is to prevent errors where we find the sum in the hash table when it is not actually
    //in the same path as the current node
    incrementHashTable(sumTable, runningSum, -1);
    
    return totalPaths;
}

// increments sum in hash table by 1 or -1
function incrementHashTable(sumTable, key, delta) {
    let sum = sumTable[key] === undefined ? 0 : sumTable[key];
    let newCount = sum + delta;
    if(newCount === 0) {
        delete SumTable[key];
    }
    else {
        sumTable[key] = newCount;
    }
}

## Invert a Binary Tree

In [10]:
class TreeNode {
    constructor(data) {
        this.data = data;
        this.left = null;
        this.right = null;
    }
}

var r = new TreeNode('A');
var B = new TreeNode('B');
var C = new TreeNode('C');
var D = new TreeNode('D');
var E = new TreeNode('E');
var F = new TreeNode('F');
var G = new TreeNode('G');

r.left = B;
r.right = C;
B.left = D;
B.right = E;
C.left = F;
C.right = G;

function preOrder(node) {
    if(node !== null) {
        console.log(node.data);
        preOrder(node.left);
        preOrder(node.right); 
    }
}

preOrder(r);

function invertTree(node) {
    if(node !== null) {
        [node.left, node.right] = [node.right, node.left];
        invertTree(node.left);
        invertTree(node.right);
    }
}

invertTree(r);

console.log('\n')
preOrder(r);

A
B
D
E
C
F
G


A
C
G
F
B
E
D


<a class="anchor" id="bits"></a>
# Bit Manipulation

### General Stuff:
* bits can either be a 1 or a 0
* there are 8 bits in 1 byte: so 1111 1111 = 8 bits = 1 byte
* 1 byte can store 1 character, e.g. 'A' or 'x' or '#'
* n bits yilds 2^n different patterns of 1s and 0s
    - 8 bits = 2^8 = 256 different numbers (0 -> 255)
* bytes and characters
    - ASCII = encoding representing each typed character as a number
        - each number stored in 1 byte (0 --> 255)
        - ex: A is 65, B is 66, a is 97, and space = 32
        can convert A + 32 = a
        - supports 128 characters only
        - 7 bits to rep a character
        - requires less space
    - unicode = encoding for mandaring, greek, arabic, etc
        - typically 2-bytes (16-bit) per character but can usually use 8-bit or 32-bit as well
        - requires more space
        - can support a wide variety of characters for various languages
* integers can be stored with either 4 or 8 bytes by doing this equation: 
    - can calculate the amount of numbers for each by doing this equation -2^(n - 1) --> 2^(n - 1) - 1
    - positive integers have 1 less number b/c it also accounts for 0
* Tricks:
    - Represent a negative number using Two's Complement
        1. get binary rep of absolute vaue of the number
        2. get complement of number
        3. add 1 to it
    - another way:
        1. get absolute value of number
        2. subtract by 1
        3. get binary representation of it
        4. negate it
    - bit subtraction:
        1. negatie subtractor
        2. add 1 to it
        3. add two numbers together
* JavaScript Representation:
    - can type out binary numbers like this:
        - let a = 0b11111111
        - this is binary form of 255
        - can also do octal and hex if you replace the b with an o or an x
        - 0xFF = 255
        - 0o377 = 255
* bitwise operators(simple):
    - bitwise operators treats operands as a sequence of 32 bits (4 bytes)
    - AND: a & b
        - 1 & 1 = 1
        - 1 & 0 = 0
    - OR: a | b
        - 1 | 0 = 1
    - XOR: a ^ b
        - 1 ^ 0 = 1
        - 1 ^ 1 = 0
        - 0 ^ 0 = 0
    - NOT: ~a
        - ~(101) = 010
* Bitwise Operators(shifting):
    - left shift: a << b
        - a << b is essentially a * 2^b
        - shifts all digits in binary representation of a to the left b number of times
        - shifts in 0s from the right
    - arithmetic right shift: a >> b
        - a >> b is essentially a / 2^b
        - shifts all digits in binary represention of a to the right b number of times
        - shifts in 0s from the left
        - keeps the sign of the number so if it is a negative number, it will still be negative, i.e. the most significant bit(sign bit) is carried over
    - logical right shift: a >>> b
        - shifts a to the right b number of times and shifts in 0s from the left
        - the difference between logical and arithmetic right shift is that the sign bit is not carried over
        - the sign bit is instead replaced with a 0, so the result is always non-negative
    - repeated arithmetic right shift would make a negative number = -1 while repeated logical right shift would make a negative number = 0
### Things to Ask:

### [Go back to top](#top)

In [1]:
var bin = 0b11111111;
var decimal = 255;
console.log(bin === decimal);

true


In [2]:
var a = 0b1101;
var b = 0b1001;
console.log({ a, b});
console.log('\n');

var and = a & b;
var or = a | b;
var xor = a ^ b;
var not = ~a;

console.log({and, bit: and.toString(2)});
console.log({or, bit: or.toString(2)});
console.log({xor, bit: xor.toString(2)});
console.log({not, bit: not.toString(2)});

{ a: 13, b: 9 }


{ and: 9, bit: '1001' }
{ or: 13, bit: '1101' }
{ xor: 4, bit: '100' }
{ not: -14, bit: '-1110' }


In [3]:
var a = -12;
var b = 2;

var leftShift = a << b;
var rightShift = a >> b;
var LogicalrightShift = a >>> b;

console.log({leftShift});
console.log({rightShift});
console.log({LogicalrightShift})

{ leftShift: -48 }
{ rightShift: -3 }
{ LogicalrightShift: 1073741821 }


In [13]:
/* Get Bit: gets bit at the ith position
1. left shift 1 by i to create a mask: 1 << i
2. num & mask, then see if it is equal to 0
    - the left shift moves 1 to the ith position. so if we have 1 << 4, then we have: 00010000;
    - then when we num & mask, the ith bits are compared. then we return whether that value is not equal to 0
*/

function getBit(num, i) {
    let mask = 1 << i;
    let value = num & mask;
    // true = 1, false = 0
    return value !== 0;
}

var a = 0b00010000; // 16
getBit(a, 4); //returns true = 1;

true

In [15]:
/* Set Bit: sets bit at the ith position:
1. create a mask by left shifting 1 by i: 1 << i
2. then we do num | mask;
    - setting = make the bit a 1 while clearing = make a bit a 0
    - since 1 | any # is always gonan be 1, when we num | mask, we
    will keep other bits the same but the ith bit will always be 1 b/c
    ith bit in the mask is 1
*/

function setBit(num, i) {
    let mask = 1 << i;
    return num | mask;
}

var a = 0b10100001; // 161
// setting bit at 4th postion is like adding 16 to it
var newA = setBit(a, 4);
console.log(newA); // 177
console.log(getBit(newA, 4)); // true = 1

177
true


In [18]:
/* Clear Bit:
1. create a mask by left shifting 1 by i then getting its complement
    - reason why we do this is b/c left shifting 1 will create something like this:
    00010000 where there is only one 1 in the mask
    - when we get the complement, we essentially have one 0 and all ones in the number: 111011111
2. then we do num & mask
    - with the complement, this essentially will clear the ith bit of num b/c the ith bit
    in the mask is 0 and the rest are 1s
    - since the rest are 1s and we num & mask, the 1s in the num will be maintained so
    the num won't change beisdes that bit
    - this is the only time we use the NOT (~) operating for clearing a bit. we will be using
    it for another operation though
*/

function clearBit(num, i) {
    let mask = ~(1 << i);
    return num & mask;
}

var a = 0b10110001; // 177
// clearing bit at pos 4 is like subtracting by 16
var newA = clearBit(a, 4);
console.log(newA); // 161
console.log(getBit(newA, 4)); // false = 0

161
false


In [19]:
/* Clear Bit from Most Significant Bit (MSB) to i (inclusive):
1. create a mask by left shifting 1 by i then subtracting it by 1
    - when we subtract by 1, the bits from i - 1 --> 0 are turned into 1 whereas the bits
    from msb --> i are still 0s
    - ex:
        - if we have a mask 0001 0000 and we subtract by 1, we are essentially adding by -1
        - 1111 1111 = -1
        - thus when we add them together, we get 1 0000 1111
        - the extra 1 on th left gets discarded so we have 0000 1111
2. then we do num & mask which would clear bits from msb --> i and keep i - 1 --> 0 bits the same
*/

function clearMsbToI(num, i) {
    let mask = (1 << i) - 1;
    return num & mask;
}

var a = 0b11110000; // 240
// we essentially cleared out those four 1s at the beginning
// since those were the only bits that added any value, clearing them
// made the number become 0
var newA = clearMsbToI(a, 4);
console.log(newA);

0


In [21]:
/* Clear Bit from i to 0 (inclusive):
1. create a mask by left shifting by -1 by i + 1
    - the reason why we want to use -1 is b/c -1 = 1111 1111
    - when we left shift something, we move every bit to the left and shift a 0 from the right
    - in doing so, we essentially replace all 1s from i --> 0 with 0s
    - for example, for i = 4, we left shift it by 5 times: 1111 1111 --> 1111 1110 --> 1111 1100
    --> 1111 1000 --> 1111 0000 --> 1110 0000
2. then we do num & mask which would mask bits from i --> 0 while keeping msb --> i + 1 the same
*/

function clearZeroToI(num, i) {
    let mask = -1 << (i + 1);
    return num & mask;
}

var a = 0b00011111; // 31
// we essentially cleared out the five 1s here
// since they were the only ones that had any value
// clearing them out turned the number to 0
var newA = clearZeroToI(a, 4);
console.log(newA);

0


In [24]:
/* Update Bit: update the bit at ith position with a given value
1. clear the bit at i
    - create mask by left shifting 1 by i then getting its complement
    - then you do num &= mask so that it clears the ith bit in num
2. then you create another mask by shifting the given value by i
3. then you do num | value mask
    - so if the value bit is 0, then the num will stay 0 since it was cleared
    - if the value bit is 1, we know that 1 | 0 = 1, so it'll be set
*/

function updateBit(num, i , bitIs1) {
    let value = bitIs1 ? 1 : 0;
    let mask = ~(1 << i);
    num &= mask;
    value <<= i;
    return num | value;
}

var a = 0b00000000;
// updating bit at the 5th pos with a 1
// since our original value is 0, the 5th bit will ony be the one
// providing it value
// 2 ^ 5 = 32
var newA = updateBit(a, 5, 1);
console.log(newA);

var b = 0b11111111; // 255
// clearing the 4th bit from b is like subtracting 16 from it
var newB = updateBit(b, 4, 0);
console.log(newB);

32
239


In [ ]:
// Powers of 2
/*
* any number that is a power of 2 will only have one bit in it
    - so 4 = 0100
* by subtracting it by 1, we get 4 - 1 = 3 = 0011
* when we do x & (x - 1) we get 1000 & 0111
    - if it is a power of 2, then they would cancel each other out and be a zero value
    - if it is not a power of 2, there would be a 1 remaining, causing it to be a non-zero value
*/
function isPowerofTwo(x) {
    return (x & (x - 1)) === 0;
}

console.log(isPowerOfTwo(65));
console.log(isPowerOfTwo(128));

In [27]:
// Count the Number of Ones in the Binary Representation of the Given Number
/*
* same concept as power of 2 algorithm
* it flips every bit from the rightmost 1-bit in n to the 0th position
* and by masking it, we clear all those one bits from the original number
* we continually do this until we reach the last 1-bit in n
* to which it would be cleared and the value of the number becomes 0
*/

function countOnes(n) {
    let count = 0;
    while(n !== 0) {
        n &= n - 1;
        count++;
    }
    return count;
}

var num = 0b11111111;
console.log(countOnes(num));

8


In [29]:
// Generate All Possible Subsets of a Set
/*
* basically just reading flags
* if we have a set {a, b, c, d} then each of these represents a bit position in a number
* since # of subsets = 2^set.length, we can represent each unique set as a binary representation of each number from 0 to (set^2) - 1
    - so since set.length = 4, we have 16 = 1111
    - 1111 = {a, b, c, d} are all present and 0000 = {} empty set
* 1 << n = 2^sets.length
* the middle for-loop checks every bit position
    - 1 << j is the mask and j = 0...n - 1
    - so if we have n = 4, j = 0, 1, 2, 3 bit positions
    - and we do i & (1 << j) to get the bit and see if it is set
*/

function possibleSets(set) {
    let n = set.length;
    
    // for let i < number of possibles subsets
    // possible subsets = (set.length) ^ 2
    for(let i = 0; i < (1 << n); i++) {
        console.log('{ ');
        
        for(let j = 0; j < n; j++) {
            if( (i & (i << j) ) > 0) {
                console.log(set[j] + ' ');
            }
        }
        
        console.log('}\n')
    }
}

possibleSets(['a','b','c']);

{ 
}

{ 
a 
}

{ 
a 
}

{ 
a 
b 
}

{ 
a 
}

{ 
a 
c 
}

{ 
a 
b 
}

{ 
a 
b 
c 
}



In [30]:
// Find the Largest Power of 2 that is Less Than or Equal To the Given Number N
/*
* essentially the same logic as counting the number of 1-bits in the binary sequence
* we eliminate the rightmost bit until the original number is 0
    - but we keep the value of the number before it reaches 0
    - that is what x is for
*/

function largestPower(n) {
    let x;
    while(n !== 0) {
        x = n;
        console.log({x});
        n &= (n - 1);
    }
    return x;
}

largestPower(14);

{ x: 14 }
{ x: 12 }
{ x: 8 }


8

In [32]:
// Highest Power of 2 that Divides a Given Number
/*
* basically, you're just isolating the rightmost bit of the number and returning it
* you can get the rightmost bit by doing:
    1. x ^ (x & (x - 1));
    2. x & (-x) // what the algorithm is doing
*/

// 192 = 11000000
// 191 = 10111111
// -191 = 01000000
// 192 & 191 = 01000000 = 64
function highestPowerOf2(n) {
    let mask = ~(n - 1);
    return n & mask;
}

highestPowerOf2(192);

64

## 5.1 Insertion
* You are given two 32-bit numbers, N and M, and two bit positions, i and j. Write a method to insert M into N such that M starts at bit j and ends at bit i. You can assume that the bits j through i have enough space to fit all of M. That is, if M = 10011, you can assume that there are at least 5 bits between j and i. You would not, for example, have j = 3, an i = 2, because M could not fully fit between bit 3 and 2.
* example:
    - input: N = 10000000000, M = 1001, i = 2, j = 6

In [33]:
function updateBits(n, m, i, j) {
    // want mask to look like 11110000011;
    // where the 0s clear out the bit where M is supposed to be
    let allOnes = ~0;
    
    // this would create 11110000000
    let left = allOnes << (j + 1);
    
    // this would create 00000000011;
    let right = ((1 << i) - 1);
    
    // left | right = 111100000011;
    let mask = left | right;
    
    // cleared out the bits from j --> i in n
    let n_cleared = n & mask;
    // then shifted the bits from m to match up with j --> ith positions
    let m_shifted = m << i;
    
    // n | m to put m into n
    return n_cleared | m_shifted;
}

var N = 0b10000000000;
var M = 0b10011;

updateBits(N, M, 2, 6).toString(2);

'10001001100'

<a class="anchor" id="ood"></a>
## Object-Oriented Design

### General Stuff:
* Approach:
    1. Handle Ambiguity:
        - use the 6 Ws: who, what, when, where, how, why
        - should ask clarifying questions b/c these interview questions can be vague
    2. Define Core Objects:
        - core objects used in a system
        - ex: design a restaurant
            - tables, guests, party, order, meal, employee, server, host
    3. Analyze Relationships:
        - see how the core objects are related to each other
        - are some objects members of others?
        - do they inherit from others?
        - one-many or many-many relationships?
        - ex: party = array of guests, server/host inherit from employees, each table has 1 party, 1 host at a restaurant, etc
    4. Investigate Actions:
        - what actions does each object take and how do they relate to other objects?
        - ex: party walks into restaurant, guest requests a table, etc

### Things to Ask:

### [Go back to top](#top)

## 7.1 

## 7.2

## 7.5

## 7.7

## 7.11

## 7.12

<a class="anchor" id="dp"></a>
## Recursion and Dynamic Programming

### General Stuff:
* the 3 laws of recursion:
    1. must have a base case
        - a condition that allows a recursive algorithm to stop
    2. must change its state and move towards the base case
        - usually by modifying some data
    3. must call itself, recursively
        - this is what makes something recursive
* apply dynamic programming to optimization problems
    - finding some minimum or maximum value where many possible solutions are viable
* apply 4 steps to develop a dp algorithm
    1. characterize structure of an optimal solution
    2. recursively define the value of an optimal solution
    3. compute the value of an optimal solution, usually in a bottom-up fashion
    4. construct an optimal solution from computed information
* basically, you want to see if the problem can be solved by breaking it up into subproblems and the accumulation of these solutions leads you to an optimal solution
    - you test out each solution and reuse any previously calculated ones to speed up time
* 2 approaches:
    - top down memoization
        - use of recursion to iterate through all subproblems
        - and then keep track of the previously calculated soln in a hash table or array
    - bottom-up iterative
        - iterative solution that builds from a small subproblem
* always figure out how to do it in the naive, recursive way first
    - then once you have that, you can optimize it further by memoizing it, i.e. keeping previous answers in a hash table to look up later
    - being able to figure out the relationship between a problem and its subproblems is key
* if you are stuck on a problem, do he base-case and build approach
    - once you have a base case for n = 0, n = 1, n = 2, n = 3, etc, then you can start to see a pattern
    - an n = 4 solution will depend on a modified n = 3 solution and so on
    - then just write out some pseudo code for the general structure of the algorithm and the steps you want to accomplish
    - your base case will help you
* if a problem talks about arrays being sorted or unsorted, think about binary search
    - you could possibly modify the original binary search algorithm to solve the entire problem
    - if not, it could still help you think about breaking up the problem recursively
* if a problem talks about finding n-ways of representing an n-value, think about fibonacci
    - recursive calls will be func(n - some value, ways)
    - then to optimize it, you would always have a table, memo {}, that keeps already solved values that can then be returned for subsequent calls
* always go with your instinct and trust the process
    - recursion can be tricky
    - sometimes, you just gotta trust that it magically works

### [Go back to top](#top)

In [2]:
// Fibonacci
/*
* goal: compute nth fibonacci number
* Fn = Fn-1 + Fn-2
* F0 = 0
* F1 = 1
*/

// naive recursive algorithm
// O(2^n)
function fib(n) {
    if(n <= 2) {
        return n;
    }
    else {
        return fib(n - 1) + fib(n - 2);
    }
}

// optimized dp algorithm
// O(n)
var memo = {};
function memoFib(n , memo) {
    if(n <= 2) {
        return n;
    }
    else if (memo[n]) {
        return memo[n];
    }
    else {
        memo[n] = memoFib(n - 1, memo) + memoFib(n - 2, memo);
        return memo[n];
    }
}

## 8.1 Triple Step
* a child is running up a staircase with n steps and can hop either 1 step, 2 step, or 3 steps at a time. Implement a method to count how many possible ways the child can run up the stairs.

In [3]:
// similar pattern and algorithm to fibonacci
/*
* naive rec. alg: O(3^n)
* memoized soln: O(n)
    - b/c the only time you have a recursive call is when you have to
    figure out the value of a new number
    - else, it'll be in the memo table already
*/

// naive recursive algorithm
function stepCount(n) {
    return n <= 0 ? 0 : _stepCount(n, n, 0);
}

function _stepCount(n, current, ways) {
    if(current === 0) {
        ways++;
        return ways;
    }
    else if (current < 0) {
        return ways;
    }
    else {
        ways = _stepCount(n, current - 1, ways);
        ways = _stepCount(n, current - 2, ways);
        ways = _stepCount(n, current - 3, ways);
    }
    return ways;
}

In [4]:
// memoized version

function stepMemo(n) {
    let memo = {};
    return n <= 0 ? 0 : _stepMemo(n, n, 0, memo);
}

function _stepMemo(n, current, ways, memo) {
    if(memo[current]) {
        return ways + memo[current];
    }
    else if (current === 0) {
        ways++;
        return ways;
    }
    else if (current < 0) {
        return ways;
    }
    else {
        ways = _stepMemo(n, current - 1, ways, memo);
        ways = _stepMemo(n, current - 2, ways, memo);
        ways = _stepMemo(n, current - 3, ways, memo);
        memo[current] = ways;
    }
    return ways;
}

## 8.2 Robot in Grid
* imagine a robot sitting on the upper left corner of grid with r rows and c columns. the robot can only move in two directions, right and down, but certain cells are 'off limits' such that the robot cannot step on them. design an algorithm to find a path for the robot from the top left to the bottom right

In [6]:
// recursive algo
/* O(2 ^(r + c)) solution
1. if maze is empty or null, return null
2. call on _getPath(maze, r, c, path)
    - if path is an array of points(r, c)
3. first checks if row or cols are out of bounds, then checks if the spot can be stepped on
4. if it can, then check if it is at the origin
5. if it is not, move down or right by 1 point
6. once it is at the origin, push the current (r, c) point to the path array
7. return true;

* overlapping problem:
    - for a path to [r, c], we check [r - 1, c] or [r, c - 1]
    - then we check the adjacent cells of those two so [r - 2, c], [r - 1, c - 1], [r - 1, c - 1], [r, c - 2]
    - as you can see, we check [r - 1, c - 1] twice so that is unnecessary work that can be avoided and instead memoized
*/
function getPath(maze) {
    if(maze == null || maze.length === 0) {
        return null;
    }
    let path = [];
    if(_getPath(maze, maze.length - 1, maze[0].length - 1, path)) {
        return path;
    }
    return null;
}

function _getPath(maze, r, c, path) {
    if(c < 0 || r < 0 || !maze[r][c]) {
        return false;
    }
    
    let isAtOrigin = (r === 0) && (c === 0);
    
    // this helps check if there even IS a path
    // b/c if it is not at the origin and both _getPath() calls return false,
    // we know there is no way for a path to form
    if(isAtOrigin || _getPath(maze, r, c - 1, path) || _getPath(maze, r - 1, c, path)) {
        path.push({r, c});
        return true;
    }
    return false;
}

In [7]:
// memoized version
/* O(r * c)
1. same as before but we have a hash table called failedPoints
2. if the cell is a dead-end, and we have to backtrack, add the rc into failedPoints
3. and if w have already arrived at the cell before, i.e. failedPoints[rc] === true, then
we return false and don't check that spot again
*/
function getPathDP(maze) {
    if(maze == null || maze.length === 0) {
        return null;
    }
    let path = [];
    let failedPoints = {};
    if(_getPathDP(maze, maze.length - 1, maze[0].length - 1, path, failedPoints)) {
        return path;
    }
    return null;
}

function _getPathDP(maze, r, c, path, failedPoints) {
    if(c < 0 || r < 0 || !maze[r][c]) {
        return false;
    }
    
    // if we already visited this point
    if(failedPoints[`${r}${c}`]) {
        return false;
    }
    
    let isAtOrigin = (r === 0) || (c === 0);
    
    if(isAtOrigin || _getPathDP(maze, r, c - 1, path, failedPoints) || _getPathDP(maze, r - 1, c, path, failedPoints)) {
        path.push({r, c});
        return true;
    }
    failedPoints[`${r}${c}`] = true;
    return false;
}

## 8.3 Magic Index
* a magic index in an array A[0...n - 1] is defined to be an index such that A[i] = i. Given a sorted array of distinct integers, write a method to find a magic index, if one exists, in array A
* Follow-Up: what if the values are not distinct

In [ ]:
// since they asked about sorted arrays and follow-up asked about non-sorted arrays
// try to see if it is related to binary search somehow

/* Binary search method:
* when A[mid] < mid, we know that the magic index cannot be on the left side and must be
on the right side
* this is b/c if A[mid] is already small, then moving from i --> i - 1 would have the values
also decrease by 1+
* since the array is sorted as well, these values will always be small
* this applies if A[mid] > mid too but all the values on the right side, i --> i + 1, would
be too big
* will this work with non-distinct values? No
    - we can't really conclude which side the magic index could be on
    - thus, we should compare midIndex and midValue (value that is at the middle of the entire array) for equality first, then:
        - left: search from start --> Math.min(midIndex - 1, midValue)
        - right: search from Math.max(midIndex + 1, midValue) --> end
*/

function magicFast(array) {
    return _magicFast(array, 0, array.length - 1);
}

function _magicFast(array, start, end) {
    if(end < start) {
        return -1;
    }
    let mid = Math.trunc((start + end) / 2);
    if(array[mid] === mid) {
        return mid;
    }
    else if (array[mid] > mid) {
        return _magicFast(array, start, mid - 1);
    }
    else {
        return _magicFast(array, mid + 1, end);
    }
}

In [ ]:
// Follow-Up for non-distinct elements

function magicFast(array) {
    return _magicFast(array, 0, array.length - 1);
}

function _magicFast(array, start, end) {
    if(end < start) {
        return -1;
    }
    
    let midIndex = Math.trunc((start + end) / 2);
    let midValue = array[midIndex];
    if(midValue === midIndex) {
        return midIndex;
    }
    
    // search left
    let leftIndex = Math.min(midIndex - 1, midValue);
    let left = _magicFast(array, start, leftIndex);
    if(left >= 0) {
        return left;
    }
    
    // search right
    let rightIndex = Math.max(midIndex + 1, midValue);
    let right = _magicFast(array, rightIndex, end);
    
    return right;
}

## 8.4 Power Set
* write a method to return all subsets of a set

In [11]:
// iterative solution with bits

function powerSet(set) {
    let pSet = [];
    let max = 1 << set.length;
    
    for(let i = 0; i < max; i++) {
        let subset = convertInt(i, set);
        pSet.push(subset);
    }
    return pSet;
}

function convertInt(num, set) {
    let subset = [];
    for(let i = 0; i < set.length; i++) {
        // creates a mask that looks at the bit at the ith position
        let mask = 1 << i;
        // then if the bit at the ith position in our num
        // is set, then that means that the element
        // in set[i] is present, so we add that to the subset
        if( (num & mask) > 0) {
            subset.push(set[i]);
        }
    }
    return subset;
}

powerSet(['a', 'b', 'c']);

[
  [],
  [ 'a' ],
  [ 'b' ],
  [ 'a', 'b' ],
  [ 'c' ],
  [ 'a', 'c' ],
  [ 'b', 'c' ],
  [ 'a', 'b', 'c' ]
]

In [12]:
function returnSubsets(set) {
    let subsets = [];
    const recurse = (currSet, remainingSet) => {
        subsets.push(currSet);
        for(let i = 0; i < remainingSet.length; i++) {
            recurse(currSet.concat([remainingSet[i]]), remainingSet.slice(i + 1));
        }
    };
    
    // the empty array is used to hold the remainingSet values
    // to create the subsets
    recurse([], set);
    return subsets;
}

## 8.6 Towers of Hanoi
* in the classic problem of the Towers of Hanoi, you have 3 towers and N disks of different sizes which can slide onto any tower. The puzzle starts with disks sorted in ascending order of size from top to bottom (i.e. each disk sits on top of an even larger one). You have the following constraints:
    1. only one disk can be moved at a time
    2. a disk is slid off the top of one tower onto another tower
    3. a disk cannot be placed on top of a smaller disk. 
* write a program to move the disks from the first tower to the last using stacks

In [ ]:
/* Base case and build approach
* cases:
    - n = 1: move disk 1 from tower 1 to tower 3
    - n = 2:
        1. move d1: t1 --> t2
        2. move d2: t1 --> t3
        3. move d1 from t2 --> t3
    - n = 3:
        1. move top 2: t1 --> t2
        2. move d3: t1 --> t3
        3. move top 2: t2 --> t3
    - n = 4
        1. move top 3: t1 --> t2
        2. move d4: t1 --> t3
        3. move tpo 3: t2 --> t3
* essentially, you know that you can move (n - 1) disks to a buffer, then move the last
disk to the last tower, then move the rest to the last tower

pseudo code:

moveDisks(n, origin, destination, buffer)
    if (n <= 0) return
    
        // move top n - 1 disks from tower 1 to tower 2
        // origin --> buffer using destination as intermediary
        moveDisks(n - 1, origin, buffer, destination)
        
        // then move the nth disk from tower 1 to tower 3
        moveTop(origin, destination)
        
        // then move the top n - 1 disks from tower 2 to tower 3
        // buffer --> destination using origin as intermediary
        moveDisks(n - 1, buffer, destination, origin)
*/

class Towers {
    constructor(n) {
        this.towers = [ [], [], []];
    }
    
    solve(n) {
        for(let i = n; i > 0; i--) {
            this.towers[0].push(i);
        }
        console.log({before: this.towers});
        this.moveTo(n, this.towers[0], this.towers[1], this.towers[2]);
    }
    
    moveTo(n, origin, buffer, destination) {
        if(n > 0) {
            this.moveTo(n - 1, origin, destination, buffer);
            this.moveTop(origin, destination);
            this.moveTo(n - 1, buffer, origin, destination);
        }
    }
    
    moveTop(origin, destination) {
        destination.push(origin.pop());
    }
}

var hanoi = new Towers();
hanoi.solve(4);
console.log(hanoi.towers);

## 8.7 Permutations without Dups: 
* write a method to compute all permutations of a string of unique characters

In [17]:
/* base case and build approach
1. if given a string a1,a2,a3, we would first create permutations of a1
2. then for each permutation of a1, we would put in a2 in all possible locations of the string
3. then for all permutations of a1a2, we would then put a3 into all possible locations of the string
4. so essentially: perm(a1,a2,a3) => a1 + perm(a2, a3) => a2 + perm(a3) => a3 + perm(''), 
where perm('') = ''

ex:
'' => 'a' => 'ab', 'ba' => 'cab', 'acb', 'abc', 'cba', 'bca', 'bac'
*/

function permUnique(str, perms = []) {
    // base case
    if(str === '') {
        perms.push('');
        return perms;
    }
    // get list of perms for single letter, then double letters, etc
    else {
        perms = permUnique(str.slice(1), perms);
        let newPerms = [];
        let first = str[0];
        
        // for every permutation in the list
        // put the current letter into the permutation at all positions
        // for example: 'a' + ['bc'] = 'abc', 'bac', 'bca'
        for(let i = 0; i < perms.length; i++) {
            let word = perms[i];
            for(let j = 0; j <= word.length; j++) {
                let newWord = spliceLetter(word, j, first);
                newPerms.push(newWord);
            }
        }
        return newPerms;
    }
}

function spliceLetter(word, index, letter) {
    word = word.split('');
    word.splice(index, 0, letter);
    word = word.join('');
    return word;
}

var perms = permUnique('abcd');
console.log({
    perms,
    len: perms.length
})

{
  perms: [
    'abcd', 'bacd', 'bcad',
    'bcda', 'acbd', 'cabd',
    'cbad', 'cbda', 'acdb',
    'cadb', 'cdab', 'cdba',
    'abdc', 'badc', 'bdac',
    'bdca', 'adbc', 'dabc',
    'dbac', 'dbca', 'adcb',
    'dacb', 'dcab', 'dcba'
  ],
  len: 24
}


## 8.8 Permutations with Dups: 
* write a method to compute all permutations of a string whose characters are not necessarily unique. The list of permutations should not have duplicates

In [ ]:
/*
1. keeps track of the length of the original string
2. if the length matches the permutation's length, add it to the hash table
3. then if we see that same word again, skip it
*/

function permDups(str) {
    let dups = {}; // hash table of duplicate permutations
    let perms = [];
    // also keeps track of the original string's length
    return _permDups(str, str.length, dups, perms);
}

function _permDups(str, len, dups, perms) {
    if(str === '') {
        perms.push('');
        return perms;
    }
    else {
        perms = _permDups(str.slice(1), len, dups, perms);
        let newPerms = [];
        let first = str[0];
        
        for(let i = 0; i < perms.length; i++) {
            let word = perms[i];
            for(let j = 0; j <= word.length; j++) {
                let newWord = spliceLetter(word, j, first);
                // if newWord is not a duplicate,
                // add it to the newPerms list
                if(dups[newWord] === undefined) {
                    newPerms.push(newWord);
                }
                // if the new word is a permutation
                // add it to the dups table
                if(newWord.length === len) {
                    dups[newWord] = true;
                }
            }
        }
        return newPerms;
    }
}

function spliceLetter(word, index, letter) {
    word = word.split('');
    word.splice(index, 0, letter);
    word = word.join('');
    return word;
}

In [19]:
/*
* faster than my solution on average for letters with duplicates like 'aaaaaaaa'
1. count the number of occurrences of each letter in the string
2. then decide to use one of these letters as the first string and the remainder will be recursed on
    - so if we have 'aabc', we pick 'c' as our first, then we do permsDup('aab');
    - then we choose 'a' as our first, then recurse for permsDup('ab');
    - and so on until we have no chars left
*/

function printPerms(str) {
    let result = [];
    let map = buildFreqTable(str);
    _printPerms(map, '', str.length, result);
    return result;
}

function buildFreqTable(str) {
    let map = {};
    for(let i = 0; i < str.length; i++) {
        let char = str[i];
        if(map[char] === undefined) {
            map[char] = 1;
        }
        else {
            map[char]++;
        }
    }
    return map;
}

function _printPerms(map, prefix, remaining, result) {
    // base case where perms are completed
    if(remaining === 0) {
        result.push(prefix);
        return;
    }
    
    // try remaining letters for next char, and generate remaining permutations
    
    // for every char in the map
    // if the char's count > 0, then push it
    for(let char in map) {
        let count = map[char];
        if(count > 0) {
            map[char]--;
            console.log({
                prefix,
                char,
                new: prefix + char
            });
            _printPerms(map, prefix + char, remaining - 1, result);
            map[char]++;
        }
    }
}

console.log(printPerms('aabc'));

{ prefix: '', char: 'a', new: 'a' }
{ prefix: 'a', char: 'a', new: 'aa' }
{ prefix: 'aa', char: 'b', new: 'aab' }
{ prefix: 'aab', char: 'c', new: 'aabc' }
{ prefix: 'aa', char: 'c', new: 'aac' }
{ prefix: 'aac', char: 'b', new: 'aacb' }
{ prefix: 'a', char: 'b', new: 'ab' }
{ prefix: 'ab', char: 'a', new: 'aba' }
{ prefix: 'aba', char: 'c', new: 'abac' }
{ prefix: 'ab', char: 'c', new: 'abc' }
{ prefix: 'abc', char: 'a', new: 'abca' }
{ prefix: 'a', char: 'c', new: 'ac' }
{ prefix: 'ac', char: 'a', new: 'aca' }
{ prefix: 'aca', char: 'b', new: 'acab' }
{ prefix: 'ac', char: 'b', new: 'acb' }
{ prefix: 'acb', char: 'a', new: 'acba' }
{ prefix: '', char: 'b', new: 'b' }
{ prefix: 'b', char: 'a', new: 'ba' }
{ prefix: 'ba', char: 'a', new: 'baa' }
{ prefix: 'baa', char: 'c', new: 'baac' }
{ prefix: 'ba', char: 'c', new: 'bac' }
{ prefix: 'bac', char: 'a', new: 'baca' }
{ prefix: 'b', char: 'c', new: 'bc' }
{ prefix: 'bc', char: 'a', new: 'bca' }
{ prefix: 'bca', char: 'a', new: 'bcaa' }


<a class="anchor" id="system"></a>
## System Design and Scalability

### Handling the Questions:
* communicate: always ask the interviewer some clarifying questions about the problem
* go broad first: try to get the bigger picture before diving into specifics
* use the whiteboard: draw a digram of your designs to help visualize it for yourself and for the interviewer
* acknowledge interview concerns: take the concerns into consideration for the designs and adjust accordingly
* be careful about assumptions: incorrect assumptions can change the way you design the solution
* state your assumptions explicitly: the interview can correct your assumptions and lead you to a better solution
* estimate when necessary: depending on the problem, making estimations will help you scale the problem later on
* drive: keep asking questions, keep making assumptions, and keep on communicating with your interviewer

### Design: Step-By-Step:
1. scope the problem
    - want to be sure that we are designing something that the interview wants
    - so asking clarifying questions heps a ton
    - so if you were designing TinyURL, you would ask about:
        - what TinyURL does
        - is it automatically generated or can a user specify the url
        - are there any analytics for it
        - how long should it last for
        - and what is the url associated with the tinyurl
2. make reasonable assumptions
    - designs for TinyURL might be different between whether it processes 100 or 100 million URLs a day
3. draw the major components:
    - this is essentially the whiteboarding section where you want to identify the major players in your design
    - for example, you might have a couple of servers that handle web crawling or keeping track of analytics
    - should have an idea of what happens when user's interact with it or how it stores data
        - so the the TinyURL example, you'll have to sketch out what happens when a user enters in a new URL
4. identify the key issues:
    - figuring out any challenges or bottlenecks to the design
    - for example, some URLs will be more heavily accessed than others so you want to handle that type of scenario
5. redesign for the key issues:
    - modify the designs to solve those issues
    - so for the heavily accessed links, they can be cached so that you don't have to look into the database over and over again
    - constantly communicate with the interview on the redesigns as well

### Algorithms that Scale: Step-By-Step:
1. ask questions:
    - always do this to understand the full scope of the problem and what exacty the interview wants
2. make believe:
    - make an assumption that the solution can fit on one machine and there are no memory limitations
    - this will help you outline a solution early on
3. get real:
    - examine how your solution would work if those same memory limitations were in place
    - are there any issues with it?
4. solve problems:
    - now you wan to be able to solve the problems presented in step 3
    - you don't really need to solve all the problems b/c new ones can arise
    - it is better to be able to take a step back and analyze and call out the problems and be able to provide some sort of a solution

### Key Concepts:
* horizontal vs vertical scaling:
    - vertical scaling: increasing the resources for a specific node. for example, adding additional memory to a server
        - easier but limited
    - horizontal scaling: increasing number of nodes. so you could have additional servers rather than having 1 server handle it all
* load balancer:
    - distributes the load evenly for a website by using multiple servers. thus if one server goes down, the whole system won't collapse as well
    - for example: having multiple servers with the same data but all the requests are distributed evenly between them all rather than just one server
* database denormalization and NoSQL:
    - joins in a large SQL database can be quite slow
    - by denormalizing it, you add redundant data to related tables so that it does not require joins to retrieve it
        - for example, if you're retrieving various tasks for a project and there are multiple projects, you can add the project name to the Tasks table and retrieve that with the tasks rather than joining the Projects table with the Tasks table
    - by using a NoSQL database, you can also achieve this as well by keeping the Project name and the task together
        - either by having a Projects document with an array of tasks
        - or a collection of Tasks documents with a Project Name field
* Database Partitioning (Sharding):
    - sharding: splitting data across multiple mchines with a way of retrieving those pieces from those machines
    - common ways of partition:
        - vertical partitioning: partitioning by feature
            - so if you had a social media network, you would have a database for profiles, another for messages, etc
                - drawback: if one of those databases gets too large, you will have to partition it as well and you might have to use another partitioning method to do so
        - key-based (or hash-based) parititioning: uses some part of the data to partition it, like an ID
            - you have N number of servers and put data on modulo(key, n)
            - drawback: your number of servers, N, will be fixed and adding additional servers would have you rehashing everything which is very expensive
        - directory-based partitioning: maintain a lookup table where all of the data can be found
            - makes it easy to add additional servers to manage the load
            - drawbacks:
                1. lookup table is a single point of failure. so if it fails, everything else will as well since you won't be able to find anything
                2. this is also a bottleneck as well if there are a lot of requests
    - many places uses a mix of these partitioning schemes
* caching:
    - a simple key-value pairing that helps you rreturn results very quickly
    - you store some frequently accessed data into a cache and anytime it is requested, you check the cache and return it
* asynchronous processing and queues:
    - slow operations should be done asynchronously to prevent the user from waiting
        - so the operation can be done in the background while the user does other things
    - for example: if you were on reddit, it could provide you with some old posts and stuff and then fetch for the newer ones in the background while you browse. then once it is finished, you are able to look at those new posts
    - or when facebook uploads a really larger video as a status update, it will do it in the background (asynchronously) while you browse. then once it is done, it will notify you and you can now see it on the timeline
* networking metrics:
    - bandwith: max amount of data transferred per unit of time
        - e.g. 8mb of data per second can be transferred
    - throughput: the amount of data that can be transferred
        - so throughput could be like 8gb of data or something
    - latency: how long it takes data to go from one end to the other
        - e.g. in fighting games, you have 2 frames of lag or 4 frames of lag
* mapreduce:
    - used to process large amounts of data and can be done in parallel
    - has 2 steps:
        1. map step: takes in data and converts it to <key, value> pairs
        2. reduce step: takes those key-value pairs and condenses them somehow into new key-value pairs
    - for example, if you have multiple documents and you want to find the certain words for it, you can use map-reduce to make key-value pairs of all the words as the key and where it occurs as the value
        - so if 'book' occurs in documents 3, 4, and 5, then the key-value pair is 'book': {doc3, doc4, doc5}
        - the map step will involve processing multiple documents like this in parallel for each word
        - the reduce step will then combine all the occurrences of the word into one
            - so if you have 'book': {doc3, doc4, doc5} in one node and 'book': {doc6, doc7}, then reduce will combine that into 'book':{doc3, doc4, doc5, doc6, doc7}


### Considerations:
* failures: any part of the system can fail so plan accordingly for them
* availability and reliability:
    - availability: % of time that system is operational
    - reliability: probability that the system is operational for a certain amount of time
* read-heavy vs write-heavy:
    - for read-heavy, you can cache some of the results to return them quickly
    - for write-heavy, you can queue the writes but if it fails, you should have a back-up plan
* Security:
    - know what kind of security risks can exist out there and plan your designs with them in mind

### There is no "perfect" system
* your goal is to look at the bigger picture and understand what needs to be done
* making smart assumptions based on what you want to achieve and analyze the designs for any drawbacks or issues

### [Go back to top](#top)

## 9.1 Stock Data:
* imagine you are building some sort of service that will be called by up to 1000 client applications to get simple end-of-day stock price information (open, close, high, low). you may assume that you already have the data, and you can store it in any format you wish. how would you design the client facing service that provides the information to client applications? you are responsible for development, rollout, and ongoing monitoring and maintenance of the feed. describe the different methods you considered and why you would recommend your approach. your service can use any technologies you wish, and can distribute the information to the client applications in any mechanism you choose.

* want to focus on how to distribute info to clients:
* need to think about these various aspects of the solution:
    - client ease of use: should be easy to implement and be useful for clients
    - ease for ourselves: needs to be easy for us to implement and should be mindful of the cost of implementation and maintenance
    - flexibility for future demands: solution should not be too constrained and should be flexible enough to handle any changes in demand
    - scalability and efficiency: solution should be efficient enough for us without sacrificing ease of use

***
Proposal #1:
* could keep data in text files of some kind so that clients could download it
* easily maintained since info could be backed up and viewed
* disadvantages: 
    - would have to require parsing this text file to respond to any queries
    - client would have to parse these files somehow and if the structure of the file changes, then the parsing mechanism could break;

***
Proposal #2:
* could use a standard SQL database and let clients query from that
* benefits:
    - very easy to query for any sort of data
    - rollbacks, backing up data, and security are a standard part so we don't have to do it ourselves
    - really easy for the client to integrate this into their apps b/c a SQL database is so widely used
* disadvantages:
    - a SQL database is kind of overkill for what we have to do since there aren't really many complex queries or other features that we need for a simple news feed
    - difficult to read the data so need some sort of way to convert it to something readable so that's an implementation cost
    - need to be careful about how much access we allow clients to have
    - clients can perform expensive and inefficient queries that we have to pay for

***
Proposal #3:
* using XML to distribute information
    - great b/c it has a fixed format and size
* advantages:
    - easy to distribute and can easily be read by both humans and machines
        - XML is a standard data model to share and distribute data b/c of this
    - most languages have libraries that can parse XML really easily so clients don't have to do much to implement this
    - can add additional data to the XML file without breaking the clients' parsing mechanism
    - can use existing tools for backing up data in XML files and we don't need to implement our own
* disadvantages:
    - no way for clients to only query for the parts that they want. they get ALL of the info even if they don't need it so it is inefficient
    - any queries would require parsing of the entire XML file

***
* could provide a web service for client data access
    - it's a bit of work on our end to implement but it adds some more security and it can be easier for the client to integrate it to their app
    - clients will have to adhere their queries to how we want though so this is both a pro and a con. clients will have to adhere to our guidelines but clients will be very limited if they have unique queries

## 9.2 Social Network:
* How would you design the data structures for a very large social network like Facebook or LinkedIn? Describe how you would design an algorithm to show the shortest path between two people (e.g. Me -> Bob -> Susan -> Jason -> You).

* Step 1: simplify the problem - forget about the millions of users
    - can construct a graph by treating each person as a node and letting edge between nodes indicate that they are friends
    - finding a path between two people could be accomplished with bfs with one of the people as the first node
    - dfs would not work well b/c it would keep going deeper and deeper down one person's connections even if the two people are separated by 1 mutual
    - alternatively, can do a bidirectional breadth-first search where we start at both of the users until the searches collide at a point
        - this is the fastest method
        - but it does require having both a source node and a destination node which is not always the case
* Step 2: handle the millions of users
    - for a service the size of Facebook, data must be kept on multiple machines
    - so each user must have a list of their friends as user IDs and we must traverse each machine for a specific id
    - so the traversal is:
        1. for each friend ID: let machine_index = getMachineIDForuser(personID);
        2. go to machine #machine_index
        3. on that machine: let friend = getPersonWithID(person_id)

***
* optimization: reduce machine jumps:
    - jumping from one machine to another is epxensive so just look up all the friends we need to on one machine then move onto the next instead of looking for each one at a time
* optimization: smart division of people and machines:
    - should divide people by their backgrounds so that people with similar backgrounds are on the same machine rather than having random people in different machines
    - so should divide by country, city, state, etc
* question: breadt-first search usually requires 'marking' a node as visited. how do you do that in this case?
    - since multiple searches can be going on at the same time, marking nodes isn't going to work
    - should instead have some sort of a hash table to keep that info and look it up if we want to check if it's been visited

## 9.3 Web Crawler:
* if you were designing a web crawler, how would you avoid getting into infinite loops?

* how does an infinite loop occur?
    - the web can be viewed as a graph of links and therefore if there is a cycle in this graph, then an infinite loop can occur while crawling
* to prevent infinite loops:
    - we need to detect any cycles
    - can be done by creating a hash table where hash[v] = true for any page, v, that has already been visited
* thus when we crawl a web using bfs, we visit each page, put the links into a queue, and after fully visiting it, we put it into a hash table
* but what does it mean for a page to be already visited?
     - we could say that a page is visited if the URL is already in the hash table
         - this is flawed however b/c sometimes URLs can be distinct but the contents of the page can still be the same
         - this is especially the case with queries added to the URL
    - would could say that a page is visited if we have different content on it
        - but if the content is randomly generated but the page is the same, then we have entered an infinite loop by constantly crawling it
* thus a solution would be to compare the similarities between the pages we've already seen and the current page we're crawling. if they are very similar, we deprioritize crawling its children
* for example:
    1. have a db with a list of items we need to crawl. on each iteration, we pick the highest priority page to crawl then
    2. open up the page and create a signature of it based on specific subsections of the page and its URL
    3. query the db to see if the signature has been crawled recently
    4. if something comes up, reinsert this page back into the db with a low priority since we've aready seen it
    5. if not, crawl the page and insert links into the db

## 9.4 Duplicate URLs:
* you have 10 billion URLs. how do you detect the duplicate documents? in this case, assume 'duplicate' means that the URLs are identical.

* how much space does 10 billion URLs take up?
    - each URL is about 100 chars on avg and each char is 4 bytes
    - so 10 bil URLs ~= 4 terabytes
* pretend that a machine can actually hold this much data in memory
    - we can then just create a hash table and map each URL we iterate through and identify the duplicates that way
    - or we can sort through the list for diplicates
* but realistically, we would have to split up this data across multiple machines

***
Solution #1: Disk Storage:
* if all data was stored on one machine, this could be done in 2 passes:
    1. first pass splits up uRLs into 4000 chunks of 1GB each
        - this can be done by storing each URL, u, in a file named < x >.txt where x = hash(u) % 4000
        - thus the URLs with the same hash value will be in the same file
    2. in the second pass, we can just load each one into memory and create a hash table of the URLs and look for duplicates

***
Solution #2: Multiple Machines
* we can do the same procedure as solution 1 but with multiple machines instead of files
* just send the URL to some machine and create hash tables and look for duplicates
* advantages:
    - the task can be done in parallel on multiple machines
    - so looking through all 4k chunks will take less time
* disadvantage:
    - we now rely on 4000 different machines to act perfectly
    - and it is much more complex working with this many machines than just disk storage

<a class="anchor" id="sorts"></a>
## Sorting and Searching

### General Stuff:
* anything that involves searching with sorted arrays will be a binary search type question
    - just modify the original algorithm to fit the question's parameters and you're good to go
* use a binary search tree/ hash table if you need fast insertions and fast lookups
    - an example of this is problem 10.10 that asks you to read in a stream of integers and update/retrieve their rankings
    - if you use a binary search tree, you could easily get O(log n) insertions that sorts the stream of integers for you and allows for O(log n) update and retrieval of rankings

### [Go back to top](#top)

In [7]:
/* Binary Search
* only works on ordered lists
* algorithm:
    1. examine the middle item
    2. if it is larger than what we are looking for, then check arr.slice(0, middleIndex)
    3. else, check arr.slice(middleIndex + 1, arr.length)
* Analysis:
    - time complexity: O(log n)
        - b/c the algorithm continually splits the array in half every time the item isn't found
    - space complexity: O(log n) for a recursive approach but is O(1) for an iterative approach
        - this is b/c recursive calls get pushed onto a stack and there are going to be log n function calls on the stack
        - iterative approach does not implicitly use a stack, so just O(1)
*/

// recursive approach
function binarySearch(item, arr) {
    return _binarySearch(item, arr, 0, arr.length - 1);
}

function _binarySearch(item, arr, start, end) {
    // to prevent int overflow
    let mid = Math.trunc(start + ((end - start) / 2));
    if(end < start) {
        return -1;
    }
    if(arr[mid] === item) {
        return mid;
    }
    else if (arr[mid] > item) {
        return _binarySearch(item, arr, start, mid - 1);
    }
    else {
        return _binarySearch(item, arr, mid + 1, end);
    }
}

console.log('Recursive:')
var list = [3, 5, 6, 8, 11, 12, 14, 15, 17, 18];
console.log(binarySearch(3, list)); // true
console.log(binarySearch(18, list)); // true
console.log(binarySearch(9, list)); // -1

//iterative approach
function binarySearchIterative(item, arr) {
    let start = 0;
    let end = arr.length - 1;
    
    while(end >= start) {
        let mid = Math.trunc(start + ((end - start) / 2));
        if(arr[mid] === item) {
            return mid;
        }
        else if(arr[mid] > item) {
            end = mid - 1;
        }
        else if (arr[mid] < item) {
            start = mid + 1;
        }
    }
    
    return -1;
}

console.log('\n')
console.log('Iterative:')
var list = [3, 5, 6, 8, 11, 12, 14, 15, 17, 18];
console.log(binarySearchIterative(3, list)); // true
console.log(binarySearchIterative(18, list)); // true
console.log(binarySearchIterative(9, list)); // -1


Recursive:
0
9
-1


Iterative:
0
9
-1


In [8]:
/* Bubble Sort
* Algorithm:
    1. start at the beginning of the list
    2. if the current item is larger than the next item, then switch places
    3. else, move onto the next item in the list
    4. continue doing this until all items are sorted
* Analysis:
    - time complexity: O(n^2)
        - this is b/c there will be (n - 1) passes through the list to completely sort a list of size n
        - and for each pass, there are (n - 1) comparisons to be made
    - space complexity: O(1)
* Short Bubble Sort:
    - a variation on bubble sort
    - allows the algorithm to stop early if the arr is already sorted
        - it can stop early unlike other sorting algorithms which can be an advantage
*/

function bubbleSort(arr) {
    for(let i = arr.length - 1; i >= 0; i--) {
        for(let j = 0; j < i; j++) {
            if(arr[j] > arr[j + 1]) {
                // destructuring assignment
                // that allows for swapping of values
                // without a temp variable
                [arr[j], arr[j + 1]] = [arr[j + 1], arr[j]];
            }
        }
    }
    return arr;
}

function shortBubbleSort(arr) {
    let unsorted = true;
    let passesRemaining = arr.length - 1;
    while(passesRemaining > 0 && unsorted) {
        unsorted = false;
        for(let j = 0; j < passesRemaining; j++) {
            if(arr[j] > arr[j + 1]) {
                unsorted = true;
                [arr[j], arr[j + 1]] = [arr[j + 1], arr[j]];
            }
        }
        passesRemaining--;
    }
    return {arr, passesRemaining};
}

In [ ]:
/* Selection Sort
* similar to bubble sort except you make 1 exchange every sort
* Algorithm:
    1. start at index n - 1 (our i value)
    2. have a pointer pass through the entire list from 0 --> i
    3. if the pointer is placed on an item larger than the current max, then keep track of the pos of the max
    4. keep doing this for every index until you reach the ith index and then swap value at i and max
* Analysis:
    - time complexity: O(n^2)
        - exactly the same as bubble sort b/c has the same number of passes and the same number of comparisons per pass
        - in practice, this sort will be faster than bubble sort due to the smaller number of exchanges
    - space complexity: O(1)
*/

function selectionSort(arr) {
    // this for loop decrements from last index to 0
    for(let i = arr.length - 1; i > 0; i--) {
        let posMax = 0;
        /*
        * this for loop moves from 0 --> i to check for max
        * reason for j < i + 1 b/c there is a possibility that the arr[i] is the max
        for that pass so need to check that too
        */
        for(let j = 0; j < i + 1; j++) {
            if(arr[j] > arr[posMax]) {
                posMax = j;
            }
        }
        [arr[i], arr[posMax]] = [arr[poxMax], arr[i]];
    }
    return arr;
}

In [9]:
/* Insertion Sort
* has a sublist of sorted elements at the beginning of the list
and any new items are then inserted into this list
* Algorithm:
    1. assume index 0 is already sorted and start with a pointer at index 1
    2. you then compare the value at the pointer and move backwards until you reach 0
    3. if the value of the previous values are greater than the one at the pointer, then shift the previous ones up
    4. if the value of the previous is less than the one at the pointer, then put the pointer value at that position
* so essentially, you only move the pointer one at a time and you insert the value at the pointer in the correct
spot in the values before it
    - so if you're at index 10, then indices at 0 --> 9 are already sorted
    - you just shift the values at those indices up until you find the right position for the current value
* Analysis:
    - time complexity: O(n^2)
        - there are n - 1 passes to sort n items
        - and there are going to be n - 1 comparisons
    - space complexity: O(1)
    - since you only perform one exchange at the end of each pass and shifts are about 1/3 of the work of exchanges,
    it will have good performance on benchmarks compared to bubble sort
*/

function insertionSort(arr) {
    // assumes i = 0 is already sorted
    // so starts at i = 1
    for(let i = 1; i < arr.length; i++) {
        // saves currentValue to do exchange later
        let currentValue = arr[i];
        let position = i;
        
        // this loop finds the position to place the currentValue into
        // if we have found the right spot for the current value
        // then exit the loop
        while(position > 0 && arr[position - 1] > currentValue) {
            // shifts the larger item to the right
            // since we already have a var to keep track of the
            // value at i, we can just override the current position
            arr[position] = arr[position - 1];
            position--;
        }
        
        arr[position] = currentValue
    }
    return arr;
}

In [ ]:
/* Shell Sort
* improvement on insertion sort by breaking a list into smaller sublists
* sublists are not contiguous and instead rely on a gap to create a sublist that are
a certain value apart
* Algorithm:
    1. start at index 0
    2. figure out the gap that you want to create the sublist
        - in this case, we halve the length of the arr until gap is 1
        - so if arr.length = 10, then gap = 5, 2, then 1
    3. then do an insertion sort with gaps
        - normal insertion sort is just a gap of 1 for every pass
        - but with shellSort, you have large gaps then progressively smaller gaps until
        the whole list is sorted
* Analysis:
    - time complexity: between O(n) and O(n^2)
        - this is b/c the list progressively gets sorted every time and every pass gets more and more efficient
        - the gap used is also an indicator on how efficient it is
            - with the gap halving every time, it tends to be O(n^2)
            - with a gap that uses (2^k) - 1 formula, it can perform at O(n^3/2)
*/

function shellSort(arr) {
    // first gap of arr.length / 2
    let sublistCount = Math.trunc(arr.length / 2);
    
    while(sublistCount > 0) {
        // the i is used as the start index for gapInsertionSort
        // and we use sublistCount as the gap
        // the gap doesn't change in this for loop
        for(let i = 0; i < sublistCount; i++) {
            gapInsertionSort(arr, i, sublistCount);
        }
        
        // finds a new gap by dividing the current gap by 2
        sublistCount = Math.trunc(sublistCount / 2);
    }
    return arr;
}

function gapInsertionSort(arr, start, gap) {
    for(let i = start + gap; i < arr.length; i += gap) {
        let currentValue = arr[i];
        let position = i;
        
        while(position >= gap && arr[position - gap] > currentValue) {
            arr[position] = arr[position - gap];
            position -= gap;
        }
        
        arr[position] = currentValue;
    }
}

In [10]:
/*
Merge Sort:
* continually splits the array in half until the subarrays are empty or has 1 item in it
(base case)
* then once the halves are sorted, they are merged together to create a larger sorted array
* Algorithm:
    1. base case: if the length of the list is 0 or 1, then do nothing
    2. if the list length > 2, then split the list into two halves: leftHalf and rightHalf
    3. recurse on those halves until they are split into subarrays of length 1 or 0
    4. then once your two halves are small enough, merge them together
        - compare the leftHalf and rightHalf and put the smaller value from both into the list
        - increment the index of the half that you got the value from
        - then when one of the halves is empty, add the rest of that half into the original list
    5. keep repeating until the whole list is sorted
Analysis:
* time complexity: O(nlogn)
    - the first process continually splits the list in half until it has a length of 0 or 1. this is O(log n)
    - the second process merges the two halves together. since it will have to process n elements, i.e.
    compare elements of the two halves to put into the original list, this is O(n)
    - since the merge happens for every split and there are log n splits
* space complexity: O(n)
    - this is b/c you need space to store the two halves of the array every time it is split
    - it is not O(nlogn) space b/c you recurse on the leftHalf first THEN the righthalf, and
    you never do this in parallel. so by the time you start recursing on the rightHalf,
    your leftHalf function calls will have returned
    - there's also O(log n) function calls in the stack but it's not the dominant term
    so we remove it
* this particular implementation uses the slice operator so there is an O(k) added onto the
time complexity as well but this can be removed if we just used pointers and did everything in place
*/

function mergeSort(list) {
    if(list.length > 1) {
        let mid = Math.trunc(list.length / 2);
        let leftHalf = list.slice(0, mid);
        let rightHalf = list.slice(mid);
        
        mergeSort(leftHalf);
        mergeSort(rightHalf);
        
        merge(list, leftHalf, rightHalf);
    }
    return;
}

// O(n)
function merge(list, leftHalf, rightHalf) {
    let leftIndex = 0;
    let rightIndex = 0;
    let listIndex = 0;
    
    // compares items from both halves and puts the smallest
    // one into the original list
    // then we increment the index of the half that wetook the
    // smaller value from
    while(leftIndex < leftHalf.length && rightIndex < rightHalf.length) {
        // the <= makes the algorithm stabe
        // to account for duplicates!!
        if(leftHalf[leftIndex] <= rightHalf[rightIndex]) {
            list[listIndex] = leftHalf[leftIndex];
            leftIndex++;
        }
        else {
            list[listIndex] = rightHalf[rightIndex];
            rightIndex++;
        }
        // then we increment the listIndex to move on
        listIndex++;
    }
    
    // if we go through all of the elements in the rightHalf
    // then add remaining values from the leftHalf
    while(leftIndex < leftHalf.length) {
        list[listIndex] = leftHalf[leftIndex];
        leftIndex++;
        listIndex++;
    }
    
    // if we go through all of the elements in the leftHalf
    // then add remaining values from the righthalf
    while(rightIndex < rightHalf.length) {
        list[listIndex] = rightHalf[rightIndex]
        rightIndex++;
        listIndex++;
    }
}

In [ ]:
/* Quick Sort:
* Algorithm:
    1. selects a value called the pivot that helps to split the list
        - when the list is sorted, the pivot is placed at a split point which
        divides the list for subsequent calls
        - many ways to pick pivot but simplest is to pick first element in the list
    2. then the list is partitioned via the pivot
        - element < pivot = left side
        - element > pivot = right side
        - use of 2 pointers called leftmark and rightmark
        - if the leftmark > pivot, then we stop it there
        - and if the rightmark < pivot, then we stop it there and we switch the values
        at the leftmark and rightmark
        - else, leftmark increments by 1 and rightmark decrements by 1
        - when rightmark index < leftmark, we stop and the rightmark is now the split point
        to place the pivot at
    3. the split point will divide the list into 2 halves and we call quicksort
    on both of them to sort them out
        - so leftHalf = arr.slice(0, splitPoint);
        - rightHalf = arr.slice(splitPoint + 1);
* Analysis:
    - time complexity: O(nlogn)
        - partition will always split the list in half so there are log n divisions
        - finding the split point requires that all the items in the list be checked so that's
        O(n)
    - space complexity: O(log n)
        - b/c there are log n function calls on the stack due to quicksort being implemented
        recursively
    - worst case scenario: O(n^2)
        - this is the case where the chosen pivot does not split the list in half
        evenly and in fact skews the list to one side
        - usually the case when the list is already sorted and you pick one of the elements
        on the edges
    - to minimize the risk of quickSort degrading to O(n^2), you can use the Median of Three method:
        - pick 3 random indices in the array and choose the median value
*/

function quickSort(arr) {
    return quickSortHelper(arr, 0, arr.length - 1);
}

function quickSortHelper(arr, first, last) {
    if(first < last) {
        let splitpoint = partition(arr, first, last);
        
        quickSortHelper(arr, first, splitpoint - 1);
        quickSortHelper(arr, splitpoint + 1, last);
    }
    return arr;
}

function partition(arr, first, last) {
    // pick pivot value
    let pivot = arr[first];
    
    let leftmark = first + 1;
    let rightmark = last;
    
    let done = false;
    
    while(!done) {
        // will run until finds a value where leftmark > pivot
        // and when leftmark and rightmark haven't crossed
        while(leftmark <= rightmark && arr[leftmark] <= pivot) {
            leftmark++;
        }
        
        // will run until finds a value where rightmark < pivot
        // and when rightmark and leftmark haven't crossed
        while(rightmark >= leftmark && arr[rightmark] >= pivot) {
            rightmark--;
        }
        
        if(rightmark < leftmark) {
            done = true;
        }
        else {
            [arr[leftmark], arr[rightmark]] = [arr[rightmark], arr[leftmark]];
        }
    }
    
    // once the split point is found
    // we switch the pivot with the rightmark index
    [arr[first], arr[rightmark]] = [arr[rightmark], right[first]];
    
    // and we return the splitpoint,
    // which is the rightmark index
    return rightmark;
}

In [ ]:
/* Counting Sort
* assume there is some number k, where each element <= k
* when k = O(n), we can have theta(n) sorting
* will have 3 arrays:
    1. input array
    2. storage array to store # of occurrences of each value from input array
    3. output array
* Algorithm:
    1. create the storage array with length = k + 1, and fill it up with 0s
        - so if k = 5, storage array = [0, 0, 0, 0, 0];
    2. go through the input array and keep track of number of occurrences of each value
        - input array = [2, 5, 3, 0, 2, 3, 0, 3];
        - storage array would be: [2, 0, 2, 3, 0, 1];
            - two 0s
            - zero 1s
            - two 2s
            - three 3s
            - zero 4s
            - 1 5
        - essentialy, each index in the storage array represents a value in the input array
        and we count the number of times that value appears in the input
        - so storage_array[5] = 1 means that there is one 5 in the input array
    3. then we fill in the output array
        - keep track of index for storage array starting at 0
        - for each position in output array, we put in the storage array index if
        storage_array[index] > 0
            - this is b/c e know that there must be at least 1 occurrence of the storage array index in the input value
            - then once we add it in, we decrement value at storage_array[index];
        - if storage_array[index] = 0, we know there are no more occurrences of index in the
        output array so we move on
        - example: storage_array = [2, 0, 2, 3, 0 1]
            - output = [];
            - starting at index 0 for both, we see that storage_array[0] = 2, meaning
            there are 2 occurrences of the value 0 in the input array
            - so we put them both intout output: output = [0, 0]
            - and we decrement the value at storage_array for each time we put it in,
            so storage_array[0] = 0;
            - since it is now at 0, we move to storage_array[1], which also happens to be 0,
            so we move onto storage_array[2]
            - we see that storage_array[2] = 2, so there are 2 occurrences of the value 2
            in the input array
            - so output is now going to be: [0, 0, 2, 2]
            - keep doing this until we reach the end of the output array
* Analysis:
    - time complexity: O(n + k)
        - n = number of items in the input array
        - k = maximu int. value found in the input array
        - when we fill up the tempArray, it takes O(k) time b/c each index represents a value that might be found in the
        input array
        - keeping track of number of occurrences of each value in input array = O(n)
        b/c we iterate through n items
        - filling up the array, in my version, takes O(n + k) b/c we have to find the index in k
        that has non-zero value, and we do this for every position in output array which is equal to n
    - space complexity: O(n + k)
        - there are n elements in the input array
        - and our storage array has k elements, where k = largest int. value in input array
    - counting sort = STABLE
        - this means that duplicate values in input array are put in the same order in the output array
        - so for example: input = [1, 3(1), 3(2), 3(3), 2]
        - then output = [1, 2, 3(1), 3(2), 3(3)]
*/

function countingSort(input, largestInt) {
    let output = [];
    
    // fills up tempArr with 0s
    // where tempArr.length = largest integer value in the input array
    // so if largestInt = 20, then tempArr.length = 21
    // and tempArr[0:20] = 0;
    let tempArr = new Array(largestInt + 1).fill(0);
    
    // goes through the input array and tracks number of occurrences of
    // a value so if 2 occurs three times in input,
    // then tempArr[2] = 3;
    input.forEach(num => tempArr[num]++);
    
    // my version
    // start from the 0th index on both output and tempArr
    // at the 0th index, we look at the first value in tempArr that is not 0
    // so if tempArr = [2, 0, 2, 3, 0, 1], then index 0 has a 2 in it 
    // which means there are two 0s that should be in the output
    // thus output[0] = 0 and output[1] = 0;
    // afterwards, we decrement tempArr[0] every time we insert into output
    // once tempArr[0] = 0, we know there aren't anymore 0s left in the output
    // so we move to tempArr[1]. we see that tempArr[1] = 0, so we move to tempArr[2]
    // and so on until we reach tempArr[largestInt] which is the end
    let tempIndex = 0;
    
    for(let i = 0; i < input.length; i++) {
        while(tempArr[index] === 0) {
            tempIndex++;
        }
        output[i] = tempIndex;
        tempArr[tempIndex]--;
    }
    
    return output;
}

In [ ]:
/* Radix Sort
* starts from the least significant digit and sorts its way up
    - for example: 123, starts at 3, then 2, then 1
* requires a STABLE sorting algorithm, like counting sort, as a subroutine b/c a you
move from least significant digit to most significant digit, the list is partially sorted
and their order is maintained for every digit
    - important b/c if subroutine sorting algorithm is not stable, then it could become
    unsorted per digit
* assumes each element has at most d digits
    - d = the highest number of digits of an element in the array
    - ex: [1, 23, 33, 555, 1000]. largest # = 1000, and d = 4
* Algorithm:
    1. find the largest number in the array
    2. then find how many digits that number has, i.e. find d
    3. for i = 1 to d, use a stable sort to sort all elements
        - starts from least significant digit to most significant digit
* Analysis:
    - time complexity: O(d * (n + b))
        - d = highest number of digits in the largest number
        - n = number of elements in the list
        - b = base of the numbers
            - in this case, b = 10 b/c we are working in base 10 and each digit is an int.
            from 0 --> 9
            - for base 2, b = 2 b/c each digit is either a 0 or a 1
        - essentially: O(d * O(counting sort))
            - counting sort = O(n + k)
            - but in this case, k = base of the numbers
            - for a regular counting sort, the storage array goes from 0 --> highestn umber in array
            - but since we go by digit, it is just 0 --> highest number of a digit for that base
    - space complexity: O(n + b)
        - essentially, we create b number of buckets to store all the elements
        - so there are b buckets with n number of elements in all of the buckets
*/

// gets the digit at a specific index
function getNum(num, index) {
    let strNum = String(num);
    let end = strNum.length - 1;
    let foundNum = strNum[end - index];
    
    return foundNum === undefined ? 0 : foundNum;
}

// finds the largest number in the arr
// to find out the # of digits to iterate through
function largestNum(arr) {
    let largest = 0;
    
    arr.forEach(num => largest = largest < num ? num : largest);
    
    // returns the # of digits of the largest number in the array
    return String(largest).length;
}

function radixSort(arr) {
    let maxLength = largestNum(arr);
    
    // for each digit
    for(let i = 0; i < maxLength; i++) {
        // since there are only 10 possible values for each digit
        // e.g. 0, 1, 2, 3, ... 9 we only need a bucket of length 10
        let buckets = Array.from({ length: 10}, () => []);
        
        // for each element in the arr
        for(let j = 0; j < arr.length; j++) {
            // gets value at digit
            let num = getNum(arr[j], i);
            
            // add the number to appropriate bucket according to its
            // digit
            if(num !== undefined) {
                buckets[num].push(arr[j]);
            }
        };
        
        arr = buckets.flat();
    }
    
    return arr;
}

In [ ]:
/* Bucket Sort
* divides the elements of the unsorted array into groups called Buckets
* these buckets are sorted and then all of them concatenated
* Algorithm:
    1. find the largest number in the list and use it to create a divider constant
        - in this case divider = Math.ceiling((max + 1) / # buckets)
        - num buckets = 10
    2. make a pass through the list and put each element into a bucket
        - bucket it belongs to = element / divider
        - so if element = 10 and divider = 5, then 10 / 5 = 2 so element belongs
        to bucket 2
        - we take Math.floor(element / divider)
    3. go through eac bucket and sort all the elements in them
        - can use any sorting algorithm
        - we used quickSort in our case
    4. concatenate the buckets together
        - easiest way with javascript is Array.prototype.flat() which returns
        a new flattened array
        - ex: [[1,2], [], [3,4]].flat() = [1, 2, 3, 4]
            - great if we have a sparse distribution of elements
* Analysis:
    - time complexity: O(n)
        - finding max value to calculate divider = O(n)
        - putting all elements into their respective buckets = O(n)
        - sorting each bucket will also be O(n) as well since only a portion of the
        total number of elements is actually sorted and worst case scenario is 1 bucket has
        most of elements
            - ex: [[1], [7], [8], [9, 9, 9, 9, 9]]
        - worst case scenario for bucket sort: O(n^2)
            - this happens when almost all of the elements in the original array
            are in one bucket
            - ex: [[], [], [], [3], [4, 4, 4, 4, 4, 4, 4]]
    - space complexity: O(n)
        - the buckets array holds all the elements of the original array we want
        sorted so that is O(n)
        - and depending on the sorting algorithm, the space complexity is subject to change
        as well
*/

function bucketSort(arr) {
    let max = Math.max(...arr);
    // # buckets = 10
    let divider = Math.ceil( (max + 1) / 10);
    let buckets = Array.from({ length: 10}, () => []);
    
    // start putting elements in arr into buckets
    arr.forEach(num => {
        let key = Math.trunc(num / divider);
        buckets[key].push(num);
    })
    
    // then sort each bucket using any sorting algorithm
    // i used quicksort for this
    buckets.forEach(bucket => quickSort(bucket));
    
    return buckets.flat();
}

## 10.3 Search in Rotated Array
* given a sorted array of n integers that has been rotated an unknown number of times, write code to find an element in the array. you may assume that the array was originally sorted in increasing order.
* example:
    - input: find 5 in [15, 16, 19, 20, 25, 1, 3, 4, 5, 7, 10, 14]
    - output: 8 (the index of 5 in the array)

In [ ]:
/*
* modification of binar ysearch
* Solution:
    1. if left side is normally ordered
        - check to see if the item is within the range of the left side
        - if it is, recurse on left side
        - else, recurse on right side
    2. else if right side is normally ordered
        - check to see if the item is within range of the right side
        - if it is, recurse on right side
        - else, recurse on left side
    3. else if arr[left] = arr[mid], meaning that the left or right half is all repeated
        - check to see if the right half has any repeats
            - if it doesn't, then recurse on it
        - else if it also has repeats, then check both halves
            - first check the left half
            - if it is not successful (=== -1), then check the right half
            - else, just return the result from the left half
* time complexity: O(log n)
    - this is b/c it is just a modified version of binary search
    - and if all the elements are unique
* worst case scenario: O(n)
    - this happens when there are a bunch of duplicates that result in us having to
    search for both halves
*/

function search(arr, left, right, item) {
    let mid = Math.trunc( (left + right) / 2);
    // found item
    if(item === arr[mid]) {
        return mid;
    }
    
    // cannot find item
    if(right < left) {
        return -1;
    }
    
    /* Either the left or right half must be normally ordered.
    * Find out which side is normally ordered, then use that half
    to figure out which side to find the item.
    */
    
    // left half is normally ordered
    if(arr[left] < arr[mid]) {
        // search left if item is between left and mid
        if(item >= arr[left] && item <arr[mid]) {
            return search(arr, left, mid - 1, item);
        }
        // else search right
        else {
            return search(arr, mid + 1, right, item);
        }
    }
    // if right half is normally ordered
    else if (arr[mid] < arr[left]) {
        // search right side
        if(item > arr[mid] && item <= arr[right]) {
            return search(arr, mid + 1, right, item);
        }
        // else search left side
        else {
            return search(arr, left, mid - 1, item);
        }
    }
    // left or right half is all repeats
    else if (arr[left] === arr[mid]) {
        // if right side is different, then search it
        if(arr[mid] !== arr[right]) {
            return search(arr, mid + 1, right item);
        }
        // else we have to search both halves
        else {
            // search left half
            let result = search(arr, left, mid - 1, item);
            // if left half is not successful search right half
            // else, return the result
            return result === -1 ? search(arr, mid + 1, right, item) : result;
        }
    }
    return -1;
}

## 10.9 Sorted Matrix Search: 
* given an M x N matrix in which each row and each column is sorted in ascending order, write a method to find an element

In [ ]:
/*
* we know that:
    - we can split the matrix into 4 quadrants:
        - top-left
        - top-right
        - bottom-left
        - bottom-right
    - if we have a value called mid, then:
        - all values in the top-left quadrant including itself, will have
        a value <= to it
        - and all values in the bottom-right quadrant, including itself, will
        have a value >= to it
* Solution:
    1. find the mid of the matrix by using row and col length
    2. then the mid will be useful to find the exact row and col index
        - row = Math.floor(mid / col_length);
        - col = mid % col_length;
    3. then check matrix[row][cl] for our value
        - if it is equal, return {row, col}
        - if value < matrix[row][col], then recurse on start --> mid
        - else if value > matrix[row][col], then recurse on mid + 1 --> end
*/

function sortedMatrixSearch(matrix, value, front, back) {
    if(matrix === undefined) {
        return 'No matrix found';
    }
    
    let m = matrix.length;
    let n = matrix[0].length;
    
    if(front === undefined && back === undefined) {
        front = 0;
        back = m * n;
    }
    
    if(front > back) { return -1; }
    
    let mid = Math.trunc((front + back) / 2);
    let row = Math.trunc(mid / n);
    let col = mid % n;
    
    if(matrix[row][col] === value) {
        return {row, col};
    }
    else if (value < matrix[row][col]) {
        return sortedMatrixSearch(matrix, value, front, mid);
    }
    else {
        return sortedMatrixSearch(matrix, value, mid + 1, back);
    }
}

## 10.10 Rank from Stream
* imagine you are reading in a stream of integers. periodically, you wish to be able to look up the rank of a number x (the number of values less than or equal to x). Implement the data structures and algorithms to support these operations. that is, implement the method track(int x), which is called when each number is generated and the method getRankOfNumber(int x), which returns the number of values less than or equal to x (not including x itself)
* example:
    - stream (in order of appearance): 5, 1, 4, 4, 5, 9, 7, 13, 3
    - getRankOfNumber(1) = 0
    - getRankOfNumber(3) = 1
    - getRankOfNumber(4) = 3

In [ ]:
/*
* Solution:
    1. essentially, as the stream of ints comes in, we would be adding it
     to a binary search tree
         - each node in the tree contains info on the number of duplicates it has and its ranking
         - when we add a new number that goes into a node's left subtree, we would update all nodes
     in its right subtree by 1 since there is 1 more number that they are greater than
    2. when we want to get the rankings, we just traverse the bst normally and return the rank info
* time complexity: O(log n)
    - this is for insertion and for updating the ranks if the number goes through the left
    subtree and you have numbers on the right
    - O(log n) for finding a number's ranking as well
* worst case scenario: O(n)
    - this is when the bst is not balanced and you would essentially have to
    traverse through all the elements to find a right position or to find the element
*/

function incrementRightSubtree(node) {
    if(node !== null) {
        this.incrementRightSubtree(node.left);
        node.payload++;
        this.incrementRightSubtree(node.right);
    }
}

function put(key, val) {
    if(this[root] !== null) {
        this._put(key, val, this[root]);
    }
    else {
        this[root] = new TreeNode(key, val);
    }
    this.size++;
}

function _put(key, val, currentNode) {
    if(key < currentNode.key) {
        // if key is less than currentNode then increment its payload
        // and also increment every item in its right subtree b/c there is 1 more
        // item that they are greater than
        currentNode.payload++;
        if(currentNode.hasRightChild()) {
            this.incrementRightSubtree(currentNode.right);
        }
        if(currentNode.hasLeftChild()) {
            this._put(key, val, currentNode.left);
        }
        else {
            // if currentkey has found a place, then its value wil lbe the current node's value - 1 - # of duplicates
            // for example: 5, 1, 4, 4, 3
            // in this case, since there are two 4s, its value would be 2 instead of 1
            // and when we add 3, we see that 3 is (3 - 1 - 1) b/c there are two 4s there
            // rank[3] would be 1 which is correct b/c the number 1 is the only one that 3 is larger than
            // rather than 2 if we did not count the duplicate in, i.e. 3 - 1
            currentNode.left = new TreeNode(key, currentNode.payload - 1 - currentNode.duplicates, null, null, currentNode);
        }
    }
    else if (key === currentNode.key) {
        // if we get the same key, just increment it since the problem said rank = # of items <= current item
        // and we increment duplicates too so that when we add a right child, it will not count the duplicate
        // for its ranking
        currentNode.payload++;
        currentNode.duplicates++;
    }
    else {
        if(currentNode.hasRightChild()) {
            this._put(key, val, currentNode.right);
        }
        else {
            currentNode.right = new TreeNode(key, currentNode.payload + 1, null, null, currentNode);
        }
     }
}


class Stream {
    constructor(stream) {
        this.stream = stream;
        this.tracker = new BinarySearchTree();
    }
    
    methodTracker() {
        this.stream.forEach(int => {
            this.tracker.put(int, 0);
        });
    }
    
    getRankOfNumber(x) {
        return this.tracker.get(x);
    }
}